In [32]:
!pip install transformers==4.40.0 accelerate
!CMAKE_ARGS="-DLLAMA_CUDA=on" pip install llama-cpp-python #양자화 구동을 위한 Llama C++ 설치

In [33]:
!huggingface-cli download MLP-KTLim/llama-3-Korean-Bllossom-8B-gguf-Q4_K_M --local-dir='./'

Fetching 6 files: 100% 6/6 [00:00<00:00, 4269.74it/s]
/content


In [34]:
from llama_cpp import Llama
from transformers import AutoTokenizer

In [37]:
import pandas as pd
import random
qa_list_new = []
df_list_new = []
columns = ['질문 개수', '제시문', '프롬프트', 'instruction', '생성 질문']

In [47]:
### 연세대 2023 인문계 학종전형 p.20
presentation1 = '''
[제시문 가] : "유행은 누구나 가는 길로 가려는 모방 욕구와 자신을 표현하고 싶은 개성 욕구를 동시에 충족시켜주는 매우 독특한 현상이다. 유행을 따름으로써 자신도 주변 사람들과 똑같이 행동하고 있다는 안도감을 얻으려는 심리, 그리고 다른 사람들과 구별되는 만족감을 얻으려는 심리가 복합적으로 얽혀 있는 것이다. 남을 따르고자 하는 욕구나 소망이 결여되는 경우, 아니면 반대로 개성을 드러내려는 욕구가 결여되는 경우 유행의 영역은 더 이상 존재하지 않게 된다. 예를 들면, 모든 개인이 제각각 특수한 존재가 되려 하고 모방을 기피하는 집단에서는 유행이 발생하지 않는다. 1390년경 피렌체에서 남성 복장의 뚜렷한 유행이 존재하지 않았던 이유는 모두가 각자 독특한 방식으로 차려입고자 했기 때문이다. 반면에 유행이 전체를 지배하게 되면, 즉 처음에는 몇몇 사람이 주도했던 일을 예외 없이 모두가 따라 하게 되면, 그것이 옷이든 음악이든 더 이상 유행이라고 부르지 않는다. 유행은 결코 현재 상태에 머물지 않으며 부단히 진행된다."

[제시문 나] : "옛것이나 선조를 추종하는 경향이 관습의 시대를 지배했다면, 새로운 것을 숭배하고 동시대인들을 모방하려는 경향이 유행의 시대를 특징 짓는다. 이제 사람들은 선조를 닮으려고 하기보다는, 주위 사람들을 닮으려 한다는 것이다. 변화에 대한 애정과 동시대인들에 대한 모방이 유행의 시대를 이끄는 두 가지 중요한 원리이다. 이 원리들에는 선조의 유산을 평가 절하하고 현재의 규범을 중요시하려는 경향이 동반된다. 옛것은 더 이상 존경해야 하는 것으로 여겨지지 않는다. 사람들은 끊임없이 새것을 탐닉하며, 같은 시대에 사는 사람들을 따라 한다. 예를 들면, 근대 유럽의 상류사회는 변화에 대한 열정에 사로잡혀 있었다. 그 열정은 최신 발명품과 이국적 문물에 대한 열망으로 타올랐다. 이탈리아, 스페인, 프랑스가 선도하는 유행을 다른 국가의 상류층 사람들이 앞다퉈 모방하였다. 그들은 새로운 것이면 무엇이든지 뒤쫓아가려고 했고, 가장 최근에 생겨난 변화를 받아들이고자 했다. 중하층 사람들은 다시 상류층의 유행을 모방하였다. 유행은 이런 식으로 사회 전체에 확산하였고, 언제나 새로운 것이 나타날 때마다 그것을 추종하는 상류층에 의해 계속해서 변화하였다.

[제시문 다] : "사람은 사회적인 존재이기 때문에 지위나 역할이 달라지면 자연히 거기에 맞는 행동양식을 하게 된다. 그 행동양식 가운데 하나가 언어 사용이다. 가령,‘엄마, 아빠’와 같은 말은 어린이들의 말이고,‘자네, 댁’과 같은 단어는 어른들의 말이라는 것도 나이에 따른 사회적 행동 양식이 반영된 것이라고 할 수 있다. 이런‘연령 단계’에 의한 언어 차이 외에도, 기성세대와 신세대, 또는 노년층 세대와 청소년층 세대처럼 ‘세대 차이’에 의한 언어 차이도 있을 수 있다. 젊은 세대는 대체로 기성세대와 비교해 볼 때 유행에 민감하고 새로운 변화를 쉽게 받아들인다. 언어에서도 젊은 세대의 이런 특성이 반영된다. 젊은 세대는 ‘반모’(반말 모드), ‘인싸’(무리에 잘 어울려 지내는 사람, 영어 ‘insider’의 의미) 등과 같은 유행어와 신조어를 만드는 주축으로, 대부분의 유행어와 신조어는 자신의 독특한 개성을 표출하려는 젊은 세대를 중심으로 사용된다. 이러한 언어의 유행 현상은 세대 변화에 따른 것이다."
'''
###[문제 1] 제시문 (가), (나), (다)에는 유행에 대한 다양한 관점이 포함되어 있다. 그관점을 비교, 분석하시오. [50점]
###[문제 2] 제시문 (라)의 조사 결과를 설명하고, 그 결과를 바탕으로 제시문 (나)와(다)의 논지를 각각 평가하시오. [50점]

###2023 연세대 학종 면접 추천형 p.17
presentation2 = '''
[제시문 가] : "인간은 타인의 시선에서 벗어나 있을 때 과연 윤리적일 수 있는가? 칸다울레스 왕이 다스리는 리디아 왕국에 기게스라는 목동이 살았다. 기게스가 양을 치고 있던 어느 날 갑자기 커다란 지진이 일어났다. 지진이 일어난 자리에는 땅이 갈라져 동굴이 생겼고, 기게스는 호기심이 생겨 갈라진 동굴 안으로 들어갔다. 그는 동굴 안에서 거인의 시체가 놓여 있는 것을 발견하였다. 시체의 손가락에는 금반지가 끼워져 있었다. 기게스는 그 반지를 빼 들고 밖으로 나왔다. 그러다 우연히 자신이 끼고 있는 반지의 흠집 난 곳을 안쪽으로 돌리면 자신은 투명 인간이 되고 바깥쪽으로 돌리면 자기 모습이 다시 보인다는 사실을 알게 되었다. 이제 남들의 시선에서 벗어나 보이지 않는 힘을 갖게 된 기게스는 자연스럽게 나쁜 마음을 먹게 되었다. 가축의 상태를 왕에게 보고하는 전령으로서 궁전에 들어간 기게스는 자신의 새로운 힘인 마법 반지를 이용하여 모습을 감춘 후, 왕비를 겁탈하고 그녀를 자기 편으로 끌어들여 왕을 암살한 뒤 스스로 왕이 되었다."

[제시문 나] : "서로 잘 아는 사람들이 소규모로 모여 사는 마을 공동체에서 개인은 대체로 합리적이고 도덕적인 성향을 보인다. 그는 다른 사람들의 말에 크게 영향받기보다는, 이성과 주관에 따라 판단하고 규범에 맞게 행동한다. 혼자 있을 때도, 여럿이 있을 때와 크게 다를 바 없이 처신한다.  그런데 19세기 이후 대도시에 인구가 밀집하고 대량생산과 소비, 그리고 대중문화가 발전하면서 대중사회가 등장한다.  이제 대중 속에서 이름 없는 한 명이 된 개인은 집단적인 분위기에 복종하고 전체의 결정을 따르라는 무언의 압력에 쉽사리 굴복한다. 대중의 일원으로서 그는 익명성 아래 자신의 욕망, 정열, 관심을 분출하고 실현한다. 이때 권력을 가진 지도자의 역할은 결정적이다. 권위적 지도자는 단순하고 선동적인 말로 대중에게 행동 방향을 제시하며, 대중은 지도자의 말을 마치 절대적인 진리인 것처럼 이해한다. 대중은 직관과 감정에 따라 권위적 지도자의 말을 무비판적으로 수용한다. 무리 속의 익명적 개인들은 쉽게 흥분하고 변덕을 부리며 열정을 드러낸다. 그러한 감정 에너지는 때로 대중이 난폭하게 폭력을 행사한다든지, 용감하게 순교를 불사하는 등의 행동을 하도록 만든다."

[제시문 다] : "도시 문명의 발전은 현대 사회의 주요 특징이다. 과학과 기술의 진전을 통해 이루어진 도시화는 익명성이 전통에 따르는 도덕규범과 오랜 대면 관계를 대신하도록 만들었다. 그런데 우리는 도시인의 생존이 도시의 잔인한 익명성 탓에 소모되고 훼손된다는 말을 자주 듣는다.
개인은 작은 시골 마을에 있을 때 이미 정해진 행동규범을 따르며, 스스로 그것을 의무로 여겼다. 그러한 규범을 어길 때 마을에서 평판이 나빠지기 때문이다. 하지만 그는 누가 누군지 알 수 없는 대도시로 나오면서부터 깊은 인간관계를 쌓을 수 없게 된다는 것이다. 이런 까닭에 대중사회 속의 도시인은 마치 정체성을 상실하고 자아를 잃어버린 채로 살아간다고 비판받기까지 한다. 그러나 이처럼 도시의 익명성이 과거 마을 공동체 시절의 인간적 교류를 사라지게 했다는 비판은 그 익명성이 지니는 독특한 이점을 보지 못한 데서 나온다. 도시의 익명성은 마을 공동체의 넌더리 나는 속박에서 벗어나는 자유의 가능성을 제공하므로 위협적이고 피폐한 것이 아니라 훨씬 더 인간적이고 해방적인 현상이다. 왜냐하면 도시 생활의 익명성 형태는 인간 삶에 필수적인 사생활을 보호하는 데 도움을 주며 도시인을 마을 생활의 부담스러운 도덕규범과 강요된 인습이라는 족쇄에서 벗어나도록 만들기 때문이다. 따라서 도시인은 이런 익명성 덕택에 과거와는 비교할 수 없을 만큼 많은 사람과 다양한 교류를 할 수 있게 되었을 뿐만 아니라 자유롭고 창의적인 생각을 펼칠 수 있게 되었다."
'''
###제시문 (가), (나), (다)에는 익명성에 대한 다양한 관점이 들어있다. 제시문 (가)와(나), 그리고 제시문 (나)와 (다)의 공통점과 차이점을 각각 논하시오. (50점)
###[문제 2] 제시문 (라)의 조사 결과를 요약하고, 제시문 (나)와 (다)를 바탕으로 그 결과를 설명하시오. (50점)

###2024 서울 p.5
presentation3 = '''
[제시문 가]"사람을 믿는 것과 사실을 믿는 것은 사뭇 다른 일이다. 다음 주에 있을 과제 발표를 준비하는 데에 있어 같은 반 친구 유진이가 당신에게 도움을 줄 것인가? 당신이 유진이가 과제를 도와줄 것이라는 사실을 믿는다면, 그것은 주변 친구들을 기꺼이 도와주었던 유진이의 평소 행동 등 증거에 바탕을 둔 것일 수 있다. 반면 과제를 도와줄 것이란 사실과 관련하여 당신이 유진이라는 사람을 믿는 것은 그와의 개인적 관계에 기반한다. 설령 유진이와 친하지 않더라도 당신은 유진이가 과제를 도와줄 것이란 사실을 믿을 수 있지만, 당신이 유진이를 믿는 것은 그에 대한 당신의 개인적 태도 없이는 성립할 수 없다. ‘믿음’을 사실에 대한 믿음에, ‘신뢰’를 사람에 대한 믿음에 한정해서 말한다면, 당신이 유진이가 과제를 도와줄 것이라고 ‘믿는’ 것과 유진이가 과제를 도와줄 것이라고 ‘신뢰하는’ 것은 같은 것이 아니다."

[제시문 나]"한편, 믿음의 기반과 신뢰의 기반의 차이는 믿었던 바가 참이 아닌 것으로 드러난 경우와 신뢰했던 바가 참이 아닌 것으로 드러난 경우에 나타나는 반응의 차이를 만든다. 오후 날씨가 맑을 것이라고 믿었지만 그렇지 않은 것으로 드러났다면, 실망스럽거나 짜증이 날 수 있다. 그리고 앞으로 날씨 예측과 관련해 더 많은 증거를 찾거나 다른 종류의 증거를 찾기도 할것이다. 반면, 절도 혐의를 받고 있는 친구가 결백을 호소하여 그가 결백하다고 신뢰했지만 그렇지 않은 것으로 드러난 경우, 우리가 느끼는 바는 단지 실망스러움이나 짜증이기보다는 배신감이다."

[제시문 다]"다른 관점에서 이야기하자면, 여행을 하다가 낯선 도시에 들러 식당을 찾아갈 때, 우리는 처음 보는 사람에게 길을 묻고 그가 일러 주는 방향으로 간다. 이때 우리는 그 사람이 어떤 사람인지 특별히 아는 바가 없고, 그가 잘 알지 못하면서 무책임하게 답했다거나 우리를 골탕 먹이기 위해 엉뚱한 방향을 알려주지 않았다는 사실을 아는 것도 아니다. 그럼에도 불구하고 그 낯선 이가 말해준 방향대로 길을 간다."

[제시문 라]"또한 다른 화자는 이렇게 말한다. 사람들이 서로를 잘 신뢰하는 사회에서는 타인의 말을 쉽게 믿어 버리고 타인의 말에 더 쉽게 속을 수도 있기 때문에 거짓이 팽배해질 수 있을 것이라 생각할 수 있다. 그러나 이 사회는 거짓이 배제되고 참이 증진되는 건강한 사회로 유지된다."

'''
###[문제 1] (가)의 내용에 기반하여 (다)의 상황이 가능한 이유를 설명하시오.
###[문제 2] (가), (나)를 바탕으로, (라)의 ‘건강한 사회’가 유지될 수 있는 이유를 설명하시오.

###2023 서울대 p.1
presentation4 = '''
[제시문 가] : "생태계가 어떻게 작동하는지 알면 알수록 많은 환경정책이 부적절하다는 사실이 드러난다. 얼핏 봐서는 상관없어 보이지만 실제로는 다른 동식물에게 유난히 큰 영향력을 미치는 종에 대해 조사하는 과정에서, 나는 친환경을 표방하는 많은 농장과 그곳의 관리체계가 빈껍데기에 불과하다는 것을 점점 더 깨닫게 되었다. 그들 농장은 많은 생물의 서식처인 나무와 관목과 죽은 나무를 잃음으로써, 물리적 구조뿐 아니라 생태계를 구성하는 다양한 종들의 관계 또한 상실했다. 그런 공간에는 생명의 거미줄이 거의 몇 줄 남아 있지 않다.

[제시문 나] : "환경 파괴와 기후 위기에 대한 경각심이 커지면서 플라스틱 빨대는 일회용품 중에서 대표적인 퇴출 대상으로 지목됐다. 하지만 플라스틱 빨대를 금지하는 정책은 빨대를 반드시 필요로 하는 사람들의 요구와 충돌한다. 빨대의 기본 형태는 오래전부터 있었지만 입구 부분이 휘어지는 플라스틱 주름 빨대는 환자들을 돕기 위해 처음 발명되었다. 플라스틱을 대체하는 친환경 빨대로 제공되는 종이 빨대, 쌀 빨대, 옥수수 전분 빨대 같은 것들은 플라스틱처럼 부드럽게 휘어지지 않아 불편하고, 뜨거운 음료에서는 쉽게 분해되므로 사용이 쉽지 않다. 플라스틱 주름 빨대를 굽은 금속 빨대 등으로 대체하는 것 역시 신체 기능이 저하된 사람들에게는 위험한 상황을 만들 수 있다. 따라서 주름 빨대를 비롯해 현대에 대량 생산되는 빨대는 부드럽고 얇은 플라스틱으로 제조되므로, 신체를 움직이기 어려운 사람들이 다른 사람의 도움 없이 음료를 마실 수 있는 유일한 방법이다.

[제시문 다] : "너희 인간들은 코로나 때문에 한 명만 죽어도 호들갑을 떨면서, 우리 동물은 수천만 마리 땅에 묻고 손을 탁탁 털더라! 자기 새끼는 끔찍이 아끼면서 남의 새끼는 끔찍하게 죽이더라! 우리의 모성애를 무시하는 당신들은 그 고매한 자식 사랑으로 무얼 했는가. 미래의 하늘에 탄소를 뿜고 미래의 땅에 분뇨 폐수 살처분 시체를 버리고 미래의 숲을 마구 베고 미래의 바다를 플라스틱으로 채운 것 말고?"
'''

###[문제 1] 환경정책을 수립할 때 유념해야 할 점에 대한 (가)와 (나)의 입장을 비교하시오.
###[문제 2] (다)의 화자를 만났을 때, (가)와 (나)의 글쓴이가 자신의 입장을 각각 어떻게 변호할지 논하시오.


new_presentation = [presentation1, presentation2, presentation3, presentation4]

In [48]:
PROMPT = '''
당신은 대한민국에서 대입 논술 면접 학원을 운영하고 있는 선생님입니다.당신의 주 업무는 제시문으로부터 새로운 질문을 생성하는 것입니다.
당신은 새로운 제시문이 주어지면, 제시문들의 내용을 파악한 후 이와 관련된 질문과 답변을 생성합니다.
또한 질문들은 각 제시문 간의 관계성을 고려해 서로 비교하거나, 연관성을 물어봐야 합니다.
제시문의 전반적인 내용에 대해 골고루 질문을 생성하세요. 질문들은 특정 내용만 다루는 것이 아닌, 제시문 전체를 구체적으로 다뤄야 합니다.

예를 들어 다음과 같은 제시문 3개가 있습니다.

###
[가]진실을 추구하지만 이야기라는 틀을 벗어날 수 없는 혼종 학문인 역사학은 인문학의 경계에 위치하면서 다른 학문보다 더 어렵기도 하고 더 쉽기도 하다. 역사가들은 원하는 정보 모두를 획득할 때까지 사료를 끊임없이 파헤치고, '사실'을 다루는 자신들의 깊이를 앞세워 여타 학문의 동료들을 괴롭히는 콧대 높은 경험주의자들이다. 이와 동시에 역사책은 흔히 이야기를 중심으로 전개되며, 가장 성공적인 역사서들은 대체로 훌륭한 소설의 속성을 일정하게 갖고 있다.
역사학의 본질적 혼종성은 과거를 재구성하는 데 있어서 사실성과 허구성 사이의 경계에 관한 논쟁의 핵심적 이유이다.

[나]크리스토퍼 브라우닝(Christopher Browning)은 1942~1943년에 걸쳐 약 38,000명의 유대인 학살 명령을 수행한 독일 101예비경찰대의 재판 기록을 통해 ‘평범한 사람들’이 학살에 가담했던 이유를 설명한다. 유대인을 죽이라는 명령을 받고 당황한 대원들에게 상관은 나이가 좀 더 많은 사람들은 임무를 수행하지 못할 것 같으면 빠져도 좋다고 말했지만, 선택의 가능성에도 불구하고 80~90%의 대원들이 대량 학살에 가담했다. 브라우닝은 사회적 관계로 인해 나약한 인간이 부당한 일을 행할 수 있다고 보았다. 순응주의, 권위에 대한 복종, 임무를 거부할 때 동료들로부터 따돌림을 당할지도 모른다는 두려움이 학살 가담의 결정적 원인이라는 것이다. 브라우닝은 무엇이 보통 사람들을 그토록 잔혹한 범죄에 가담하도록 이끌었는가를 이해하려 했던 것이고 그의 결론은 집단적 순응성의 압도적인 영향이었다.

[다]대니얼 골드하겐(Daniel Goldhagen)은 브라우닝과 동일한 사료를 검토하고 정반대 결론을 내렸다. 그의 결론은 101예비경찰대의 압도적 다수가 동료들의 압력, 복종, 혹은 자신들의 경력 때문에 학살에 가담했던 것이 아니라, 섬뜩할 정도로 냉담하고 잔인한 행동을 묘사한 기록들에서 드러나듯 유대인 학살의 적극적 욕망을 가지고 행동했기 때문이라는 것이다. 골드하겐은, 학살 가담이 내키지 않았고 자신들의 행동을 혐오했다는 대원들의 진술이 자기 변호에 불과하며, 그들은 ‘평범한 보통 사람들’이 아니라 ‘비정상적인 정치문화의 보통 사람들’이라고 보았다. 그의 명제는 단순하고 명확하다. “독일인의 반유대주의적 신념이 홀로코스트를 유발한 핵심 동인이다.” 골드하겐은 사회적 관계에 초점을 맞추기보다는 반유대주의라는 당시 독일 사회의 특수성을 문제시했다. 그의 자명한 주장은 앞선 역사가들과 달랐지만, 상당한 대중적 찬사를 받았다.
###

이 제시문 세트에서 출제된 문제는 다음과 같습니다.
###
1. (가)에서 말한 역사학에서의 허구성을 구체적으로 설명하고, (나)와 (다)에서 발견되는 허구적 요소가 각각 무엇인지 설명하시오.
2. (가)에서 말한 ‘혼종성’이 다른 학문 분야에서 어떻게 나타날 수 있는지 예를 들어 설명하시오.
###

이처럼 새로운 제시문이 주어지면 이에 대한 질문을 생성하면 됩니다.
이때 생성할 질문들은 예시 질문들과 유사한 형태이어야 합니다.
다음은 예시 질문들입니다.

["(나), (다), (라) 각각에 근거하여 (가)에 나타난 정부의 물가 변동 파악 방식의 한계점을 설명하시오."]
["(가) 또는 (나)에서 문제가 된 상황과 유사한 다른 사례를 제시하고 그 이유를 설명하시오."]
["(가)와 (나)에 나타난 독자에 대한 공통된 이해 방식을 (다)의 맥락에서 평가하시오."]
["(가)와 (나)의 밑줄 친 사례에서 관찰되는 문제점의 공통점과 차이점을 구체적으로 설명하시오.]
["독자와 저자(혹은 작가)의 관계에 관해 (가)와 (나)에 제시된 입장을 비교하시오."]
["(다)의 화자를 만났을 때, (가)와 (나)의 글쓴이가 자신의 입장을 각각 어떻게 변호할지 논하시오."]

이처럼 모든 질문은 반드시 "~하시오."로 끝나야 합니다.
'''

In [49]:
presentation_number = random.randint(0,2)
num_of_question = random.randint(3,5)

!huggingface-cli download MLP-KTLim/llama-3-Korean-Bllossom-8B-gguf-Q4_K_M --local-dir='./'


model_id = 'MLP-KTLim/llama-3-Korean-Bllossom-8B-gguf-Q4_K_M'

tokenizer = AutoTokenizer.from_pretrained(model_id, verbose = 0)

model = Llama(
    model_path='/content/llama-3-Korean-Bllossom-8B-Q4_K_M.gguf', #다운로드받은 모델의 위치
    n_ctx=5000,
    n_gpu_layers=-1, verbose = 0        # Number of model layers to offload to GPU
)

generation_kwargs = {
    "max_tokens":2000,
    "stop":["<|eot_id|>"],
    "top_p":0.9,
    "temperature":0.8,
    "echo":True, # Echo the prompt in the output
}

for i in range(4) :
  num_of_question = random.randint(3,5)
  if num_of_question == 3 :
    format = '''
    "질문 1"
    "답변 1"
    "질문 2"
    "답변 2"
    "질문 3"
    "답변 3"'''
  elif num_of_question == 4 :
    format = '''
    "질문 1"
    "답변 1"
    "질문 2"
    "답변 2"
    "질문 3"
    "답변 3"
    "질문 4"
    "답변 4"'''

  else :
    format = '''
    "질문 1"
    "답변 1"
    "질문 2"
    "답변 2"
    "질문 3"
    "답변 3"
    "질문 4"
    "답변 4"
    "질문 5"
    "답변 5"'''

  test_text = f'''다음 제시문을 보고, 질문과 답변 세트 {num_of_question}개 생성하라.

  주의점은 다음과 같다.
  ###
  1. 다른 불필요한 문장은 생성하지 마시오. 오직 질문과 답변만 생성하시오.
  2. 질문은 반드시 한국어로, 1~2문장으로 구성해야 한다.
  3. 각 질문들은 서로 다른 형태와 내용으로, 겹치지 않아야 한다.
  4. 모든 질문은 반드시 "~하시오."의 명령조 형식으로 끝나야 한다.
  5. 또한 질문은 최대한 자연스러운 말투이어야 한다.
  6. 마지막으로 질문에는, 답변에 대한 내용이 나타나서는 안 된다.
  ###

  예를 들어, 질문은
  ###
  "제시문 [가]와 [나]에 기반하여 [다]에 대한 ~~~를 설명하시오."
  "제시문 [가]를 기반으로 [나]의 ~~~~에 대해 논의하시오."
  "제시문에서 나타난 ~~~~에 대한 설명을 서술하시오."
  ###
  등의 형식이어야 한다.

  생성 형식은
  {format}
  으로 해야한다.

  새로운 제시문은 다음과 같다.
  ###
  {new_presentation[i]}
  ###
                  '''

  messages = [
    {"role": "system", "content": f"{PROMPT}"},
    {"role": "user", "content": f"{test_text}"}
    ]

  prompt = tokenizer.apply_chat_template(
    messages,
    tokenize = False,
    add_generation_prompt=True, verbose = 0)

  resonse_msg = model(prompt, **generation_kwargs)
  generated = resonse_msg['choices'][0]['text'][len(prompt):]
  print(generated)
  print(f'질문 개수 : {num_of_question}')
  qa_list_new.append(generated)
  df_list_new.append([num_of_question, new_presentation[presentation_number], PROMPT, test_text, generated])
  print('\n')

Fetching 6 files: 100% 6/6 [00:00<00:00, 2838.46it/s]
/content


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


질문 1:
제시문 [가]와 [나]에 따라 [다]에서 사용되는 '사회적인 존재'라는 용어가 어떻게 사용되고 있는지 설명하시오.

답변 1:
[다]에서는 '사회적인 존재'라는 용어가 사용되어 사람들이 사회적 존재로서 인식되고 있음을 이야기하고 있습니다. 이는 그들이 다른 사람들로부터 영향을 받으며, 그들이 사회적 역할을 수행하며, 그들이 사회적 상황에 따라 행동해야 한다는 것을 의미합니다.

질문 2:
제시문 [가]를 기반으로 [나]의 유행에 대해 논의하시오.

답변 2:
[나]의 유행은 새로운 것이 생기는 것과 변화를 쉽게 받아들이는 것입니다. 사람들은 선조를 따라하지 않고, 동시대 사람들을 모방하려고 합니다. 이는 변화에 대한 애정과 동시대인들에 대한 모방이 유행을 이끌어나가는 주요한 원리입니다. 이러한 원리는 상류사회의 경우 특수히 나타납니다. 이들은 혁신적인 기술을 쉽게 수용하고, 새로운 스타일을 적극적으로 추구합니다.

질문 3:
제시문에서 제시된 ˈ허용되지 않는 음악ˈ의 특징에 대해 설명하시오.

답변 3:
제시문 [다]에서는 '허용되지 않은 음악'이 언급되었습니다. 이러한 음악은 사회적으로 부적합한 음악으로 간주됩니다. 이는 음악의 내용이 문제 때문인 경우가 많습니다. 예를 들어, [다]에서 언급된 음악은 '속삭이는', '타악기 소리와 같은 것' 등으로 묘사되어, 단순히 기악적인 내용뿐 아니라 사회적 부적합성을 강조하기 위해 사용되었습니다.

질문 4:
제시문 [가]를 기반으로 [나]와 [다]의 비교를 통해 어떤 점이 공통적이고 어떤 점이 다른지 설명하시오.

답변 4:
[가], [나], 및 [다]는 모두 유행과 관련된 개념에 대해 이야기하고 있습니다. [가]는 유행이 모든 사람들에게 보편적으로 적용된다고 주장하며, 개인의 자유로운 선택이라고 주장합니다. 반면에 [나]는 동시대인들이 새로운 것을 따라가는 경향이 있다는 것을 강조합니다. 사람들은 변화를 쉽게 받아들이고, 새로운 것을 모방하려고 합니다. 마지막으로, [다]는 사람들이 사회적 존재로서 인식

In [50]:
df_new = pd.DataFrame(df_list_new, columns=columns)
df_new

,질문 개수,제시문,프롬프트,instruction,생성 질문
0,4,"\n[제시문 가] : ""인간은 타인의 시선에서 벗어나 있을 때 과연 윤리적일 수 있...",\n당신은 대한민국에서 대입 논술 면접 학원을 운영하고 있는 선생님입니다.당신의 주...,"다음 제시문을 보고, 질문과 답변 세트 4개 생성하라.\n\n 주의점은 다음과 같...",질문 1:\n제시문 가]와 제시문 나]를 비교하여 두 가지 제시문의 화자들이 어떤 ...
1,5,"\n[제시문 가] : ""인간은 타인의 시선에서 벗어나 있을 때 과연 윤리적일 수 있...",\n당신은 대한민국에서 대입 논술 면접 학원을 운영하고 있는 선생님입니다.당신의 주...,"다음 제시문을 보고, 질문과 답변 세트 5개 생성하라.\n\n 주의점은 다음과 같...",질문 1 : 제시문 [가]와 [나]에 대한 1번의 답변은 무엇인가요?\n답변 1 :...
2,3,"\n[제시문 가] : ""인간은 타인의 시선에서 벗어나 있을 때 과연 윤리적일 수 있...",\n당신은 대한민국에서 대입 논술 면접 학원을 운영하고 있는 선생님입니다.당신의 주...,"다음 제시문을 보고, 질문과 답변 세트 3개 생성하라.\n\n 주의점은 다음과 같...",질문 1:\n제시문 [가]와 [나]에 따라 [다]에서 ˜~를 설명하시오.\n\n답변...
3,3,"\n[제시문 가]""사람을 믿는 것과 사실을 믿는 것은 사뭇 다른 일이다. 다음 주에...",\n당신은 대한민국에서 대입 논술 면접 학원을 운영하고 있는 선생님입니다.당신의 주...,"다음 제시문을 보고, 질문과 답변 세트 3개 생성하라.\n\n 주의점은 다음과 같...",이 제시문 세트에 대한 질문 및 답변 세트를 생성한다.\n \n ### 질문 1...
4,4,"\n[제시문 가]""사람을 믿는 것과 사실을 믿는 것은 사뭇 다른 일이다. 다음 주에...",\n당신은 대한민국에서 대입 논술 면접 학원을 운영하고 있는 선생님입니다.당신의 주...,"다음 제시문을 보고, 질문과 답변 세트 4개 생성하라.\n\n 주의점은 다음과 같...",질문 1:\n제시문 가]와 [나]를 비교하여 두 글의 공통점과 차이점을 설명하시오...
5,4,"\n[제시문 가]""사람을 믿는 것과 사실을 믿는 것은 사뭇 다른 일이다. 다음 주에...",\n당신은 대한민국에서 대입 논술 면접 학원을 운영하고 있는 선생님입니다.당신의 주...,"다음 제시문을 보고, 질문과 답변 세트 4개 생성하라.\n\n 주의점은 다음과 같...",질문 1:\n제시문 [가]와 [나]에 따라 [다]에서 ˜비교적 접근성˜이 무엇인지 ...
6,5,"\n[제시문 가] : ""인간은 타인의 시선에서 벗어나 있을 때 과연 윤리적일 수 있...",\n당신은 대한민국에서 대입 논술 면접 학원을 운영하고 있는 선생님입니다.당신의 주...,"다음 제시문을 보고, 질문과 답변 세트 5개 생성하라.\n\n 주의점은 다음과 같...",질문 1:\n제시문 가]와 제시문 나]를 비교하여 두 제시문의 화자들이 어떤 측면에...
7,3,"\n[제시문 가] : ""인간은 타인의 시선에서 벗어나 있을 때 과연 윤리적일 수 있...",\n당신은 대한민국에서 대입 논술 면접 학원을 운영하고 있는 선생님입니다.당신의 주...,"다음 제시문을 보고, 질문과 답변 세트 3개 생성하라.\n\n 주의점은 다음과 같...","질문 1:\n제시문 [가]와 [나]를 기반으로, [다]의 1~2문을 어떻게 이해할 ..."
8,3,"\n[제시문 가] : ""인간은 타인의 시선에서 벗어나 있을 때 과연 윤리적일 수 있...",\n당신은 대한민국에서 대입 논술 면접 학원을 운영하고 있는 선생님입니다.당신의 주...,"다음 제시문을 보고, 질문과 답변 세트 3개 생성하라.\n\n 주의점은 다음과 같...",질문 1:\n제시문 [가]와 [나]에 따라 제시된 [다]의 화자가 왜 그의 말을 쉽...
9,5,"\n[제시문 가] : ""유행은 누구나 가는 길로 가려는 모방 욕구와 자신을 표현하고...",\n당신은 대한민국에서 대입 논술 면접 학원을 운영하고 있는 선생님입니다.당신의 주...,"다음 제시문을 보고, 질문과 답변 세트 5개 생성하라.\n\n 주의점은 다음과 같...",질문 1:\n제시문 [가]와 [나]에 따라 [다]에서 사용되는 '사회적인 존재'라는...


In [54]:
df_new.to_csv('0625 시도_23시.csv', encoding = 'utf-8')

### 여기서부터는 아님

In [ ]:
### 연세대 2023 인문계 학종전형 p.20
presentation1 = '''
"유행은 누구나 가는 길로 가려는 모방 욕구와 자신을 표현하고 싶은 개성 욕구를 동시에 충족시켜주는 매우 독특한 현상이다.
유행을 따름으로써 자신도 주변 사람들과 똑같이 행동하고 있다는 안도감을 얻으려는 심리, 그리고 다른 사람들과 구별되는 만족감을 얻으려는 심리가 복합적으로 얽혀 있는 것이다.
남을 따르고자 하는 욕구나 소망이 결여되는 경우, 아니면 반대로 개성을 드러내려는 욕구가 결여되는 경우 유행의 영역은 더 이상 존재하지 않게 된다.
예를 들면, 모든 개인이 제각각 특수한 존재가 되려 하고 모방을 기피하는 집단에서는 유행이 발생하지 않는다.
1390년경 피렌체에서 남성 복장의 뚜렷한 유행이 존재하지 않았던 이유는 모두가 각자 독특한 방식으로 차려입고자 했기 때문이다.
반면에 유행이 전체를 지배하게 되면, 즉 처음에는 몇몇 사람이 주도했던 일을 예외 없이 모두가 따라 하게 되면, 그것이 옷이든 음악이든 더 이상 유행이라고 부르지 않는다.
유행은 결코 현재 상태에 머물지 않으며 부단히 진행된다|.

다른 관점에서는, 옛것이나 선조를 추종하는 경향이 관습의 시대를 지배했다면, 새로운 것을 숭배하고 동시대인들을 모방하려는 경향이 유행의 시대를 특징 짓는다.
이제 사람들은 선조를 닮으려고 하기보다는, 주위 사람들을 닮으려 한다는 것이다.
변화에 대한 애정과 동시대인들에 대한 모방이 유행의 시대를 이끄는 두 가지 중요한 원리이다.
이 원리들에는 선조의 유산을 평가 절하하고 현재의 규범을 중요시하려는 경향이 동반된다.
옛것은 더 이상 존경해야 하는 것으로 여겨지지 않는다.
사람들은 끊임없이 새것을 탐닉하며, 같은 시대에 사는 사람들을 따라 한다.
예를 들면, 근대 유럽의 상류사회는 변화에 대한 열정에 사로잡혀 있었다.
그 열정은 최신 발명품과 이국적 문물에 대한 열망으로 타올랐다. 이탈리아, 스페인, 프랑스가 선도하는 유행을 다른 국가의 상류층 사람들이 앞다퉈 모방하였다.
그들은 새로운 것이면 무엇이든지 뒤쫓아가려고 했고, 가장 최근에 생겨난 변화를 받아들이고자 했다.
중하층 사람들은 다시 상류층의 유행을 모방하였다. 유행은 이런 식으로 사회 전체에 확산하였고, 언제나 새로운 것이 나타날 때마다 그것을 추종하는 상류층에 의해 계속해서 변화하였다.

한편,사람은 사회적인 존재이기 때문에 지위나 역할이 달라지면 자연히 거기에 맞는 행동양식을 하게 된다. 그 행동양식 가운데 하나가 언어 사용이다.
가령,‘엄마, 아빠’와 같은 말은 어린이들의 말이고,‘자네, 댁’과 같은 단어는 어른들의 말이라는 것도 나이에 따른 사회적 행동 양식이 반영된 것이라고 할 수 있다.
이런‘연령 단계’에 의한 언어 차이 외에도, 기성세대와 신세대, 또는 노년층 세대와 청소년층 세대처럼 ‘세대 차이’에 의한 언어 차이도 있을 수 있다.
젊은 세대는 대체로 기성세대와 비교해 볼 때 유행에 민감하고 새로운 변화를 쉽게 받아들인다.
언어에서도 젊은 세대의 이런 특성이 반영된다. 젊은 세대는 ‘반모’(반말 모드), ‘인싸’(무리에 잘 어울려 지내는 사람, 영어 ‘insider’의 의미) 등과 같은 유행어와 신조어를 만드는 주축으로, 대부분의 유행어와 신조어는 자신의 독특한 개성을 표출하려는 젊은 세대를 중심으로 사용된다.
이러한 언어의 유행 현상은 세대 변화에 따른 것이다."
'''
###[문제 1] 제시문 (가), (나), (다)에는 유행에 대한 다양한 관점이 포함되어 있다. 그관점을 비교, 분석하시오. [50점]
###[문제 2] 제시문 (라)의 조사 결과를 설명하고, 그 결과를 바탕으로 제시문 (나)와(다)의 논지를 각각 평가하시오. [50점]

###2023 연세대 학종 추천형 p.17
presentation2 = '''
인간은 타인의 시선에서 벗어나 있을 때 과연 윤리적일 수 있는가? 칸다울레스 왕이 다스리는 리디아 왕국에 기게스라는 목동이 살았다. 기게스가 양을 치고 있던 어느 날 갑자기 커다란 지진이 일어났다. 지진이 일어난 자리에는 땅이 갈라져 동굴이 생겼고, 기게스는 호기심이 생겨 갈라진 동굴 안으로 들어갔다. 그는 동굴 안에서 거인의 시체가 놓여 있는 것을 발견하였다. 시체의 손가락에는 금반지가 끼워져 있었다. 기게스는 그 반지를 빼 들고 밖으로 나왔다. 그러다 우연히 자신이 끼고 있는 반지의 흠집 난 곳을 안쪽으로 돌리면 자신은 투명 인간이 되고 바깥쪽으로 돌리면 자기 모습이 다시 보인다는 사실을 알게 되었다. 이제 남들의 시선에서 벗어나 보이지 않는 힘을 갖게 된 기게스는 자연스럽게 나쁜 마음을 먹게 되었다. 가축의 상태를 왕에게 보고하는 전령으로서 궁전에 들어간 기게스는 자신의 새로운 힘인 마법 반지를 이용하여 모습을 감춘 후, 왕비를 겁탈하고 그녀를 자기 편으로 끌어들여 왕을 암살한 뒤 스스로 왕이 되었다.

이제부터 화자를 바꾸어 설명해보겠다. 서로 잘 아는 사람들이 소규모로 모여 사는 마을 공동체에서 개인은 대체로 합리적이고 도덕적인 성향을 보인다. 그는 다른 사람들의 말에 크게 영향받기보다는, 이성과 주관에 따라 판단하고 규범에 맞게 행동한다. 혼자 있을 때도, 여럿이 있을 때와 크게 다를 바 없이 처신한다.  그런데 19세기 이후 대도시에 인구가 밀집하고 대량생산과 소비, 그리고 대중문화가 발전하면서 대중사회가 등장한다.  이제 대중 속에서 이름 없는 한 명이 된 개인은 집단적인 분위기에 복종하고 전체의 결정을 따르라는 무언의 압력에 쉽사리 굴복한다. 대중의 일원으로서 그는 익명성 아래 자신의 욕망, 정열, 관심을 분출하고 실현한다. 이때 권력을 가진 지도자의 역할은 결정적이다. 권위적 지도자는 단순하고 선동적인 말로 대중에게 행동 방향을 제시하며, 대중은 지도자의 말을 마치 절대적인 진리인 것처럼 이해한다. 대중은 직관과 감정에 따라 권위적 지도자의 말을 무비판적으로 수용한다. 무리 속의 익명적 개인들은 쉽게 흥분하고 변덕을 부리며 열정을 드러낸다. 그러한 감정 에너지는 때로 대중이 난폭하게 폭력을 행사한다든지, 용감하게 순교를 불사하는 등의 행동을 하도록 만든다.

한편, 다른 관점에서 이야기 해보겠다. 도시 문명의 발전은 현대 사회의 주요 특징이다. 과학과 기술의 진전을 통해 이루어진 도시화는 익명성이 전통에 따르는 도덕규범과 오랜 대면 관계를 대신하도록 만들었다. 그런데 우리는 도시인의 생존이 도시의 잔인한 익명성 탓에 소모되고 훼손된다는 말을 자주 듣는다.
개인은 작은 시골 마을에 있을 때 이미 정해진 행동규범을 따르며, 스스로 그것을 의무로 여겼다. 그러한 규범을 어길 때 마을에서 평판이 나빠지기 때문이다. 하지만 그는 누가 누군지 알 수 없는 대도시로 나오면서부터 깊은 인간관계를 쌓을 수 없게 된다는 것이다. 이런 까닭에 대중사회 속의 도시인은 마치 정체성을 상실하고 자아를 잃어버린 채로 살아간다고 비판받기까지 한다. 그러나 이처럼 도시의 익명성이 과거 마을 공동체 시절의 인간적 교류를 사라지게 했다는 비판은 그 익명성이 지니는 독특한 이점을 보지 못한 데서 나온다. 도시의 익명성은 마을 공동체의 넌더리 나는 속박에서 벗어나는 자유의 가능성을 제공하므로 위협적이고 피폐한 것이 아니라 훨씬 더 인간적이고 해방적인 현상이다. 왜냐하면 도시 생활의 익명성 형태는 인간 삶에 필수적인 사생활을 보호하는 데 도움을 주며 도시인을 마을 생활의 부담스러운 도덕규범과 강요된 인습이라는 족쇄에서 벗어나도록 만들기 때문이다. 따라서 도시인은 이런 익명성 덕택에 과거와는 비교할 수 없을 만큼 많은 사람과 다양한 교류를 할 수 있게 되었을 뿐만 아니라 자유롭고 창의적인 생각을 펼칠 수 있게 되었다.
'''
###제시문 (가), (나), (다)에는 익명성에 대한 다양한 관점이 들어있다. 제시문 (가)와(나), 그리고 제시문 (나)와 (다)의 공통점과 차이점을 각각 논하시오. (50점)
###[문제 2] 제시문 (라)의 조사 결과를 요약하고, 제시문 (나)와 (다)를 바탕으로 그 결과를 설명하시오. (50점)

presentation3 = '''
진실을 추구하지만 이야기라는 틀을 벗어날 수 없는 혼종 학문인 역사학은 인문학의 경계에 위치하면서 다른 학문보다 더 어렵기도 하고 더 쉽기도 하다.
역사가들은 원하는 정보 모두를 획득할 때까지 사료를 끊임없이 파헤치고, '사실'을 다루는 자신들의 깊이를 앞세워 여타 학문의 동료들을 괴롭히는 콧대 높은 경험주의자들이다. 이와 동시에 역사책은 흔히 이야기를 중심으로 전개되며, 가장 성공적인 역사서들은 대체로 훌륭한 소설의 속성을 일정하게 갖고 있다.
역사학의 본질적 혼종성은 과거를 재구성하는 데 있어서 사실성과 허구성 사이의 경계에 관한 논쟁의 핵심적 이유이다.

이제 다른 관점에서 이야기 해보겠다. 크리스토퍼 브라우닝(Christopher Browning)은 1942~1943년에 걸쳐 약 38,000명의 유대인 학살 명령을 수행한 독일 101예비경찰대의 재판 기록을 통해 ‘평범한 사람들’이 학살에 가담했던 이유를 설명한다. 유대인을 죽이라는 명령을 받고 당황한 대원들에게 상관은 나이가 좀 더 많은 사람들은 임무를 수행하지 못할 것 같으면 빠져도 좋다고 말했지만, 선택의 가능성에도 불구하고 80~90%의 대원들이 대량 학살에 가담했다. 브라우닝은 사회적 관계로 인해 나약한 인간이 부당한 일을 행할 수 있다고 보았다. 순응주의, 권위에 대한 복종, 임무를 거부할 때 동료들로부터 따돌림을 당할지도 모른다는 두려움이 학살 가담의 결정적 원인이라는 것이다. 브라우닝은 무엇이 보통 사람들을 그토록 잔혹한 범죄에 가담하도록 이끌었는가를 이해하려 했던 것이고 그의 결론은 집단적 순응성의 압도적인 영향이었다.

한편, 다른 화자의 발언은 다음과 같다. 대니얼 골드하겐(Daniel Goldhagen)은 브라우닝과 동일한 사료를 검토하고 정반대 결론을 내렸다. 그의 결론은 101예비경찰대의 압도적 다수가 동료들의 압력, 복종, 혹은 자신들의 경력 때문에 학살에 가담했던 것이 아니라, 섬뜩할 정도로 냉담하고 잔인한 행동을 묘사한 기록들에서 드러나듯 유대인 학살의 적극적 욕망을 가지고 행동했기 때문이라는 것이다. 골드하겐은, 학살 가담이 내키지 않았고 자신들의 행동을 혐오했다는 대원들의 진술이 자기 변호에 불과하며, 그들은 ‘평범한 보통 사람들’이 아니라 ‘비정상적인 정치문화의 보통 사람들’이라고 보았다. 그의 명제는 단순하고 명확하다. “독일인의 반유대주의적 신념이 홀로코스트를 유발한 핵심 동인이다.” 골드하겐은 사회적 관계에 초점을 맞추기보다는 반유대주의라는 당시 독일 사회의 특수성을 문제시했다. 그의 자명한 주장은 앞선 역사가들과 달랐지만, 상당한 대중적 찬사를 받았다.

'''
###(가)에서 말한 역사학에서의 허구성을 구체적으로 설명하고, (나)와 (다)에서 발견되는 허구적 요소가 각각 무엇인지 설명하시오.
###[문제 2] (가)에서 말한 ‘혼종성’이 다른 학문 분야에서 어떻게 나타날 수 있는지 예를 들어 설명하시오.

###2024 서울 p.5
presentation4 = '''
사람을 믿는 것과 사실을 믿는 것은 사뭇 다른 일이다. 다음 주에 있을 과제 발표를 준비하는 데에 있어 같은 반 친구 유진이가 당신에게 도움을 줄 것인가? 당신이 유진이가 과제를 도와줄 것이라는 사실을 믿는다면, 그것은 주변 친구들을 기꺼이 도와주었던 유진이의 평소 행동 등 증거에 바탕을 둔 것일 수 있다. 반면 과제를 도와줄 것이란 사실과 관련하여 당신이 유진이라는 사람을 믿는 것은 그와의 개인적 관계에 기반한다. 설령 유진이와 친하지 않더라도 당신은 유진이가 과제를 도와줄 것이란 사실을 믿을 수 있지만, 당신이 유진이를 믿는 것은 그에 대한 당신의 개인적 태도 없이는 성립할 수 없다. ‘믿음’을 사실에 대한 믿음에, ‘신뢰’를 사람에 대한 믿음에 한정해서 말한다면, 당신이 유진이가 과제를 도와줄 것이라고 ‘믿는’ 것과 유진이가 과제를 도와줄 것이라고 ‘신뢰하는’ 것은 같은 것이 아니다.

한편, 믿음의 기반과 신뢰의 기반의 차이는 믿었던 바가 참이 아닌 것으로 드러난 경우와 신뢰했던 바가 참이 아닌 것으로 드러난 경우에 나타나는 반응의 차이를 만든다. 오후 날씨가 맑을 것이라고 믿었지만 그렇지 않은 것으로 드러났다면, 실망스럽거나 짜증이 날 수 있다. 그리고 앞으로 날씨 예측과 관련해 더 많은 증거를 찾거나 다른 종류의 증거를 찾기도 할것이다. 반면, 절도 혐의를 받고 있는 친구가 결백을 호소하여 그가 결백하다고 신뢰했지만 그렇지 않은 것으로 드러난 경우, 우리가 느끼는 바는 단지 실망스러움이나 짜증이기보다는 배신감이다.

다른 관점에서 이야기하자면, 여행을 하다가 낯선 도시에 들러 식당을 찾아갈 때, 우리는 처음 보는 사람에게 길을 묻고 그가 일러 주는 방향으로 간다. 이때 우리는 그 사람이 어떤 사람인지 특별히 아는 바가 없고, 그가 잘 알지 못하면서 무책임하게 답했다거나 우리를 골탕 먹이기 위해 엉뚱한 방향을 알려주지 않았다는 사실을 아는 것도 아니다. 그럼에도 불구하고 그 낯선 이가 말해준 방향대로 길을 간다.

또한 다른 화자는 이렇게 말한다. 사람들이 서로를 잘 신뢰하는 사회에서는 타인의 말을 쉽게 믿어 버리고 타인의 말에 더 쉽게 속을 수도 있기 때문에 거짓이 팽배해질 수 있을 것이라 생각할 수 있다. 그러나 이 사회는 거짓이 배제되고 참이 증진되는 건강한 사회로 유지된다.

'''
###[문제 1] (가)의 내용에 기반하여 (다)의 상황이 가능한 이유를 설명하시오.
###[문제 2] (가), (나)를 바탕으로, (라)의 ‘건강한 사회’가 유지될 수 있는 이유를 설명하시오.


new_presentation = [presentation1, presentation2, presentation3, presentation4]

In [ ]:
#num_of_question = int(input("원하는 질문의 개수를 입력하세요 : "))
num_of_question = random.randint(3,5)

if num_of_question == 3 :
  format = '''
  "질문 1"
  "답변 1"
  "질문 2"
  "답변 2"
  "질문 3"
  "답변 3"'''
elif num_of_question == 4 :
  format = '''
  "질문 1"
  "답변 1"
  "질문 2"
  "답변 2"
  "질문 3"
  "답변 3"
  "질문 4"
  "답변 4"'''

else :
  format = '''
  "질문 1"
  "답변 1"
  "질문 2"
  "답변 2"
  "질문 3"
  "답변 3"
  "질문 4"
  "답변 4"
  "질문 5"
  "답변 5"'''

In [ ]:
PROMPT = '''
당신은 대한민국에서 대입 논술 면접 학원을 운영하고 있는 선생님입니다.당신의 주 업무는 제시문으로부터 새로운 질문을 생성하는 것입니다.
당신은 새로운 제시문이 주어지면, 제시문들의 내용을 파악한 후 이와 관련된 질문과 답변을 생성합니다.
제시문의 전반적인 내용에 대해 골고루 질문을 생성하세요. 질문들은 특정 내용만 다루는 것이 아닌, 제시문 전체를 구체적으로 다뤄야 합니다.

이때 생성할 질문들은 예시 질문들과 유사한 형태이어야 합니다.
다음은 예시 질문들입니다.
["환경정책을 수립할 때 유념해야 할 점에 대한, 글에서의 서로 다른 입장을 비교하시오."]
["글에 등장하는 화자들이 자신의 입장을 각각 어떻게 변호할지 논하시오."]
["제시문의 첫 단락과 마지막 단락에서의 주장의 차이점과 공통점을 설명하시오."]
["글에서 등장하는 화자들의 주장의 차이점과 공통점을 설명하시오."]

이처럼 모든 질문은 반드시 "~하시오."로 끝나야 합니다.
또한 제시문에서 사용된 단어를 사용하면 안 됩니다.
'''

In [ ]:
presentation_number = random.randint(0,3)

test_text = f'''다음 제시문을 보고, 질문과 답변 세트 {num_of_question}개 생성하라. 다른 불필요한 문장은 생성하면 안 된다.
질문은 반드시 한국어로, 1~2문장으로 구성해야 한다.
각 질문들은 서로 다른 형태와 내용으로, 겹치지 않아야 한다. 반드시 '제시문'이라는 단어를 사용하고, 다른 단어는 '대사', '제시전' 등의 단어는  사용해서는 안 된다.
모든 질문은 반드시 "~하시오."의 명령조 형식으로 끝나야 한다.

예를 들어, 질문은
###
"글에 등장하는 화자들이 자신의 입장을 각각 어떻게 변호할지 논하시오."
"제시문의 첫 단락과 마지막 단락에서의 주장의 차이점과 공통점을 설명하시오."
"글에서 등장하는 화자들의 주장의 차이점과 공통점을 설명하시오."
###
등의 형식이어야 한다.

생성 형식은
{format}
으로 해야한다.

새로운 제시문
###
{new_presentation[presentation_number]}
###
                '''

In [ ]:
!huggingface-cli download MLP-KTLim/llama-3-Korean-Bllossom-8B-gguf-Q4_K_M --local-dir='./'

model_id = 'MLP-KTLim/llama-3-Korean-Bllossom-8B-gguf-Q4_K_M'

tokenizer = AutoTokenizer.from_pretrained(model_id, verbose = 0)
model = Llama(
    model_path='/content/llama-3-Korean-Bllossom-8B-Q4_K_M.gguf', #다운로드받은 모델의 위치
    n_ctx=5000,
    n_gpu_layers=-1, verbose = 0        # Number of model layers to offload to GPU
)

messages = [
    {"role": "system", "content": f"{PROMPT}"},
    {"role": "user", "content": f"{test_text}"}
    ]

prompt = tokenizer.apply_chat_template(
    messages,
    tokenize = False,
    add_generation_prompt=True, verbose = 0
)

generation_kwargs = {
    "max_tokens":2000,
    "stop":["<|eot_id|>"],
    "top_p":0.95,
    "temperature":0.5,
    "echo":True, # Echo the prompt in the output
}

resonse_msg = model(prompt, **generation_kwargs)
generated = resonse_msg['choices'][0]['text'][len(prompt):]
print(generated)
print(f'질문 개수 : {num_of_question}')
qa_list.append(generated)
df_list.append([num_of_question, new_presentation[presentation_number], PROMPT, test_text, generated])

Fetching 6 files: 100% 6/6 [00:00<00:00, 6902.31it/s]
/content


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


질문 1:
제시문에서 언급된 문화적인 차이점에 대해 설명하십시오.

답변 1:
제시문에서 언급된 문화적인 차이점은 도시 문명과 과거 마을 사회의 차이입니다. 도시에서는 익명성이 전통에 따르는 도덕규범과 깊은 인간관계를 대신할 수 있도록 허용됩니다. 이는 도시의 익명성을 통해 개인이 자신을 자유롭게 발전시킬 수 있는 기회를 제공합니다.
질문 개수 : 5


In [6]:
### 연세대 2023 인문계 학종전형 p.20
presentation1 = '''
"유행은 누구나 가는 길로 가려는 모방 욕구와 자신을 표현하고 싶은 개성 욕구를 동시에 충족시켜주는 매우 독특한 현상이다.
유행을 따름으로써 자신도 주변 사람들과 똑같이 행동하고 있다는 안도감을 얻으려는 심리, 그리고 다른 사람들과 구별되는 만족감을 얻으려는 심리가 복합적으로 얽혀 있는 것이다.
남을 따르고자 하는 욕구나 소망이 결여되는 경우, 아니면 반대로 개성을 드러내려는 욕구가 결여되는 경우 유행의 영역은 더 이상 존재하지 않게 된다.
예를 들면, 모든 개인이 제각각 특수한 존재가 되려 하고 모방을 기피하는 집단에서는 유행이 발생하지 않는다.
1390년경 피렌체에서 남성 복장의 뚜렷한 유행이 존재하지 않았던 이유는 모두가 각자 독특한 방식으로 차려입고자 했기 때문이다.
반면에 유행이 전체를 지배하게 되면, 즉 처음에는 몇몇 사람이 주도했던 일을 예외 없이 모두가 따라 하게 되면, 그것이 옷이든 음악이든 더 이상 유행이라고 부르지 않는다.
유행은 결코 현재 상태에 머물지 않으며 부단히 진행된다|.

다른 관점에서는, 옛것이나 선조를 추종하는 경향이 관습의 시대를 지배했다면, 새로운 것을 숭배하고 동시대인들을 모방하려는 경향이 유행의 시대를 특징 짓는다.
이제 사람들은 선조를 닮으려고 하기보다는, 주위 사람들을 닮으려 한다는 것이다.
변화에 대한 애정과 동시대인들에 대한 모방이 유행의 시대를 이끄는 두 가지 중요한 원리이다.
이 원리들에는 선조의 유산을 평가 절하하고 현재의 규범을 중요시하려는 경향이 동반된다.
옛것은 더 이상 존경해야 하는 것으로 여겨지지 않는다.
사람들은 끊임없이 새것을 탐닉하며, 같은 시대에 사는 사람들을 따라 한다.
예를 들면, 근대 유럽의 상류사회는 변화에 대한 열정에 사로잡혀 있었다.
그 열정은 최신 발명품과 이국적 문물에 대한 열망으로 타올랐다. 이탈리아, 스페인, 프랑스가 선도하는 유행을 다른 국가의 상류층 사람들이 앞다퉈 모방하였다.
그들은 새로운 것이면 무엇이든지 뒤쫓아가려고 했고, 가장 최근에 생겨난 변화를 받아들이고자 했다.
중하층 사람들은 다시 상류층의 유행을 모방하였다. 유행은 이런 식으로 사회 전체에 확산하였고, 언제나 새로운 것이 나타날 때마다 그것을 추종하는 상류층에 의해 계속해서 변화하였다.

한편,사람은 사회적인 존재이기 때문에 지위나 역할이 달라지면 자연히 거기에 맞는 행동양식을 하게 된다. 그 행동양식 가운데 하나가 언어 사용이다.
가령,‘엄마, 아빠’와 같은 말은 어린이들의 말이고,‘자네, 댁’과 같은 단어는 어른들의 말이라는 것도 나이에 따른 사회적 행동 양식이 반영된 것이라고 할 수 있다.
이런‘연령 단계’에 의한 언어 차이 외에도, 기성세대와 신세대, 또는 노년층 세대와 청소년층 세대처럼 ‘세대 차이’에 의한 언어 차이도 있을 수 있다.
젊은 세대는 대체로 기성세대와 비교해 볼 때 유행에 민감하고 새로운 변화를 쉽게 받아들인다.
언어에서도 젊은 세대의 이런 특성이 반영된다. 젊은 세대는 ‘반모’(반말 모드), ‘인싸’(무리에 잘 어울려 지내는 사람, 영어 ‘insider’의 의미) 등과 같은 유행어와 신조어를 만드는 주축으로, 대부분의 유행어와 신조어는 자신의 독특한 개성을 표출하려는 젊은 세대를 중심으로 사용된다.
이러한 언어의 유행 현상은 세대 변화에 따른 것이다."
'''
###[문제 1] 제시문 (가), (나), (다)에는 유행에 대한 다양한 관점이 포함되어 있다. 그관점을 비교, 분석하시오. [50점]
###[문제 2] 제시문 (라)의 조사 결과를 설명하고, 그 결과를 바탕으로 제시문 (나)와(다)의 논지를 각각 평가하시오. [50점]

###2023 연세대 학종 추천형 p.17
presentation2 = '''
인간은 타인의 시선에서 벗어나 있을 때 과연 윤리적일 수 있는가? 칸다울레스 왕이 다스리는 리디아 왕국에 기게스라는 목동이 살았다. 기게스가 양을 치고 있던 어느 날 갑자기 커다란 지진이 일어났다. 지진이 일어난 자리에는 땅이 갈라져 동굴이 생겼고, 기게스는 호기심이 생겨 갈라진 동굴 안으로 들어갔다. 그는 동굴 안에서 거인의 시체가 놓여 있는 것을 발견하였다. 시체의 손가락에는 금반지가 끼워져 있었다. 기게스는 그 반지를 빼 들고 밖으로 나왔다. 그러다 우연히 자신이 끼고 있는 반지의 흠집 난 곳을 안쪽으로 돌리면 자신은 투명 인간이 되고 바깥쪽으로 돌리면 자기 모습이 다시 보인다는 사실을 알게 되었다. 이제 남들의 시선에서 벗어나 보이지 않는 힘을 갖게 된 기게스는 자연스럽게 나쁜 마음을 먹게 되었다. 가축의 상태를 왕에게 보고하는 전령으로서 궁전에 들어간 기게스는 자신의 새로운 힘인 마법 반지를 이용하여 모습을 감춘 후, 왕비를 겁탈하고 그녀를 자기 편으로 끌어들여 왕을 암살한 뒤 스스로 왕이 되었다.

이제부터 화자를 바꾸어 설명해보겠다. 서로 잘 아는 사람들이 소규모로 모여 사는 마을 공동체에서 개인은 대체로 합리적이고 도덕적인 성향을 보인다. 그는 다른 사람들의 말에 크게 영향받기보다는, 이성과 주관에 따라 판단하고 규범에 맞게 행동한다. 혼자 있을 때도, 여럿이 있을 때와 크게 다를 바 없이 처신한다.  그런데 19세기 이후 대도시에 인구가 밀집하고 대량생산과 소비, 그리고 대중문화가 발전하면서 대중사회가 등장한다.  이제 대중 속에서 이름 없는 한 명이 된 개인은 집단적인 분위기에 복종하고 전체의 결정을 따르라는 무언의 압력에 쉽사리 굴복한다. 대중의 일원으로서 그는 익명성 아래 자신의 욕망, 정열, 관심을 분출하고 실현한다. 이때 권력을 가진 지도자의 역할은 결정적이다. 권위적 지도자는 단순하고 선동적인 말로 대중에게 행동 방향을 제시하며, 대중은 지도자의 말을 마치 절대적인 진리인 것처럼 이해한다. 대중은 직관과 감정에 따라 권위적 지도자의 말을 무비판적으로 수용한다. 무리 속의 익명적 개인들은 쉽게 흥분하고 변덕을 부리며 열정을 드러낸다. 그러한 감정 에너지는 때로 대중이 난폭하게 폭력을 행사한다든지, 용감하게 순교를 불사하는 등의 행동을 하도록 만든다.

한편, 다른 관점에서 이야기 해보겠다. 도시 문명의 발전은 현대 사회의 주요 특징이다. 과학과 기술의 진전을 통해 이루어진 도시화는 익명성이 전통에 따르는 도덕규범과 오랜 대면 관계를 대신하도록 만들었다. 그런데 우리는 도시인의 생존이 도시의 잔인한 익명성 탓에 소모되고 훼손된다는 말을 자주 듣는다.
개인은 작은 시골 마을에 있을 때 이미 정해진 행동규범을 따르며, 스스로 그것을 의무로 여겼다. 그러한 규범을 어길 때 마을에서 평판이 나빠지기 때문이다. 하지만 그는 누가 누군지 알 수 없는 대도시로 나오면서부터 깊은 인간관계를 쌓을 수 없게 된다는 것이다. 이런 까닭에 대중사회 속의 도시인은 마치 정체성을 상실하고 자아를 잃어버린 채로 살아간다고 비판받기까지 한다. 그러나 이처럼 도시의 익명성이 과거 마을 공동체 시절의 인간적 교류를 사라지게 했다는 비판은 그 익명성이 지니는 독특한 이점을 보지 못한 데서 나온다. 도시의 익명성은 마을 공동체의 넌더리 나는 속박에서 벗어나는 자유의 가능성을 제공하므로 위협적이고 피폐한 것이 아니라 훨씬 더 인간적이고 해방적인 현상이다. 왜냐하면 도시 생활의 익명성 형태는 인간 삶에 필수적인 사생활을 보호하는 데 도움을 주며 도시인을 마을 생활의 부담스러운 도덕규범과 강요된 인습이라는 족쇄에서 벗어나도록 만들기 때문이다. 따라서 도시인은 이런 익명성 덕택에 과거와는 비교할 수 없을 만큼 많은 사람과 다양한 교류를 할 수 있게 되었을 뿐만 아니라 자유롭고 창의적인 생각을 펼칠 수 있게 되었다.
'''
###제시문 (가), (나), (다)에는 익명성에 대한 다양한 관점이 들어있다. 제시문 (가)와(나), 그리고 제시문 (나)와 (다)의 공통점과 차이점을 각각 논하시오. (50점)
###[문제 2] 제시문 (라)의 조사 결과를 요약하고, 제시문 (나)와 (다)를 바탕으로 그 결과를 설명하시오. (50점)

presentation3 = '''
진실을 추구하지만 이야기라는 틀을 벗어날 수 없는 혼종 학문인 역사학은 인문학의 경계에 위치하면서 다른 학문보다 더 어렵기도 하고 더 쉽기도 하다.
역사가들은 원하는 정보 모두를 획득할 때까지 사료를 끊임없이 파헤치고, '사실'을 다루는 자신들의 깊이를 앞세워 여타 학문의 동료들을 괴롭히는 콧대 높은 경험주의자들이다. 이와 동시에 역사책은 흔히 이야기를 중심으로 전개되며, 가장 성공적인 역사서들은 대체로 훌륭한 소설의 속성을 일정하게 갖고 있다.
역사학의 본질적 혼종성은 과거를 재구성하는 데 있어서 사실성과 허구성 사이의 경계에 관한 논쟁의 핵심적 이유이다.

이제 다른 관점에서 이야기 해보겠다. 크리스토퍼 브라우닝(Christopher Browning)은 1942~1943년에 걸쳐 약 38,000명의 유대인 학살 명령을 수행한 독일 101예비경찰대의 재판 기록을 통해 ‘평범한 사람들’이 학살에 가담했던 이유를 설명한다. 유대인을 죽이라는 명령을 받고 당황한 대원들에게 상관은 나이가 좀 더 많은 사람들은 임무를 수행하지 못할 것 같으면 빠져도 좋다고 말했지만, 선택의 가능성에도 불구하고 80~90%의 대원들이 대량 학살에 가담했다. 브라우닝은 사회적 관계로 인해 나약한 인간이 부당한 일을 행할 수 있다고 보았다. 순응주의, 권위에 대한 복종, 임무를 거부할 때 동료들로부터 따돌림을 당할지도 모른다는 두려움이 학살 가담의 결정적 원인이라는 것이다. 브라우닝은 무엇이 보통 사람들을 그토록 잔혹한 범죄에 가담하도록 이끌었는가를 이해하려 했던 것이고 그의 결론은 집단적 순응성의 압도적인 영향이었다.

한편, 다른 화자의 발언은 다음과 같다. 대니얼 골드하겐(Daniel Goldhagen)은 브라우닝과 동일한 사료를 검토하고 정반대 결론을 내렸다. 그의 결론은 101예비경찰대의 압도적 다수가 동료들의 압력, 복종, 혹은 자신들의 경력 때문에 학살에 가담했던 것이 아니라, 섬뜩할 정도로 냉담하고 잔인한 행동을 묘사한 기록들에서 드러나듯 유대인 학살의 적극적 욕망을 가지고 행동했기 때문이라는 것이다. 골드하겐은, 학살 가담이 내키지 않았고 자신들의 행동을 혐오했다는 대원들의 진술이 자기 변호에 불과하며, 그들은 ‘평범한 보통 사람들’이 아니라 ‘비정상적인 정치문화의 보통 사람들’이라고 보았다. 그의 명제는 단순하고 명확하다. “독일인의 반유대주의적 신념이 홀로코스트를 유발한 핵심 동인이다.” 골드하겐은 사회적 관계에 초점을 맞추기보다는 반유대주의라는 당시 독일 사회의 특수성을 문제시했다. 그의 자명한 주장은 앞선 역사가들과 달랐지만, 상당한 대중적 찬사를 받았다.

'''
###(가)에서 말한 역사학에서의 허구성을 구체적으로 설명하고, (나)와 (다)에서 발견되는 허구적 요소가 각각 무엇인지 설명하시오.
###[문제 2] (가)에서 말한 ‘혼종성’이 다른 학문 분야에서 어떻게 나타날 수 있는지 예를 들어 설명하시오.

###2024 서울 p.5
presentation4 = '''
사람을 믿는 것과 사실을 믿는 것은 사뭇 다른 일이다. 다음 주에 있을 과제 발표를 준비하는 데에 있어 같은 반 친구 유진이가 당신에게 도움을 줄 것인가? 당신이 유진이가 과제를 도와줄 것이라는 사실을 믿는다면, 그것은 주변 친구들을 기꺼이 도와주었던 유진이의 평소 행동 등 증거에 바탕을 둔 것일 수 있다. 반면 과제를 도와줄 것이란 사실과 관련하여 당신이 유진이라는 사람을 믿는 것은 그와의 개인적 관계에 기반한다. 설령 유진이와 친하지 않더라도 당신은 유진이가 과제를 도와줄 것이란 사실을 믿을 수 있지만, 당신이 유진이를 믿는 것은 그에 대한 당신의 개인적 태도 없이는 성립할 수 없다. ‘믿음’을 사실에 대한 믿음에, ‘신뢰’를 사람에 대한 믿음에 한정해서 말한다면, 당신이 유진이가 과제를 도와줄 것이라고 ‘믿는’ 것과 유진이가 과제를 도와줄 것이라고 ‘신뢰하는’ 것은 같은 것이 아니다.

한편, 믿음의 기반과 신뢰의 기반의 차이는 믿었던 바가 참이 아닌 것으로 드러난 경우와 신뢰했던 바가 참이 아닌 것으로 드러난 경우에 나타나는 반응의 차이를 만든다. 오후 날씨가 맑을 것이라고 믿었지만 그렇지 않은 것으로 드러났다면, 실망스럽거나 짜증이 날 수 있다. 그리고 앞으로 날씨 예측과 관련해 더 많은 증거를 찾거나 다른 종류의 증거를 찾기도 할것이다. 반면, 절도 혐의를 받고 있는 친구가 결백을 호소하여 그가 결백하다고 신뢰했지만 그렇지 않은 것으로 드러난 경우, 우리가 느끼는 바는 단지 실망스러움이나 짜증이기보다는 배신감이다.

다른 관점에서 이야기하자면, 여행을 하다가 낯선 도시에 들러 식당을 찾아갈 때, 우리는 처음 보는 사람에게 길을 묻고 그가 일러 주는 방향으로 간다. 이때 우리는 그 사람이 어떤 사람인지 특별히 아는 바가 없고, 그가 잘 알지 못하면서 무책임하게 답했다거나 우리를 골탕 먹이기 위해 엉뚱한 방향을 알려주지 않았다는 사실을 아는 것도 아니다. 그럼에도 불구하고 그 낯선 이가 말해준 방향대로 길을 간다.

또한 다른 화자는 이렇게 말한다. 사람들이 서로를 잘 신뢰하는 사회에서는 타인의 말을 쉽게 믿어 버리고 타인의 말에 더 쉽게 속을 수도 있기 때문에 거짓이 팽배해질 수 있을 것이라 생각할 수 있다. 그러나 이 사회는 거짓이 배제되고 참이 증진되는 건강한 사회로 유지된다.

'''
###[문제 1] (가)의 내용에 기반하여 (다)의 상황이 가능한 이유를 설명하시오.
###[문제 2] (가), (나)를 바탕으로, (라)의 ‘건강한 사회’가 유지될 수 있는 이유를 설명하시오.


new_presentation = [presentation1, presentation2, presentation3, presentation4]

In [7]:
#num_of_question = int(input("원하는 질문의 개수를 입력하세요 : "))
num_of_question = random.randint(3,5)

if num_of_question == 3 :
  format = '''
  "질문 1"
  "답변 1"
  "질문 2"
  "답변 2"
  "질문 3"
  "답변 3"'''
elif num_of_question == 4 :
  format = '''
  "질문 1"
  "답변 1"
  "질문 2"
  "답변 2"
  "질문 3"
  "답변 3"
  "질문 4"
  "답변 4"'''

else :
  format = '''
  "질문 1"
  "답변 1"
  "질문 2"
  "답변 2"
  "질문 3"
  "답변 3"
  "질문 4"
  "답변 4"
  "질문 5"
  "답변 5"'''

In [8]:
PROMPT = '''
당신은 대한민국에서 대입 논술 면접 학원을 운영하고 있는 선생님입니다.당신의 주 업무는 제시문으로부터 새로운 질문을 생성하는 것입니다.
당신은 새로운 제시문이 주어지면, 제시문들의 내용을 파악한 후 이와 관련된 질문과 답변을 생성합니다.
제시문의 전반적인 내용에 대해 골고루 질문을 생성하세요. 질문들은 특정 내용만 다루는 것이 아닌, 제시문 전체를 구체적으로 다뤄야 합니다.

이때 생성할 질문들은 예시 질문들과 유사한 형태이어야 합니다.
다음은 예시 질문들입니다.
["환경정책을 수립할 때 유념해야 할 점에 대한, 글에서의 서로 다른 입장을 비교하시오."]
["글에 등장하는 화자들이 자신의 입장을 각각 어떻게 변호할지 논하시오."]
["제시문의 첫 단락과 마지막 단락에서의 주장의 차이점과 공통점을 설명하시오."]
["글에서 등장하는 화자들의 주장의 차이점과 공통점을 설명하시오."]

이처럼 모든 질문은 반드시 "~하시오."로 끝나야 합니다.
또한 제시문에서 사용된 단어를 사용하면 안 됩니다.
'''

In [9]:
presentation_number = random.randint(0,3)

test_text = f'''다음 제시문을 보고, 질문과 답변 세트 {num_of_question}개 생성하라. 다른 불필요한 문장은 생성하면 안 된다.
질문은 반드시 한국어로, 1~2문장으로 구성해야 한다.
각 질문들은 서로 다른 형태와 내용으로, 겹치지 않아야 한다. 반드시 '제시문'이라는 단어를 사용하고, 다른 단어는 '대사', '제시전' 등의 단어는  사용해서는 안 된다.
모든 질문은 반드시 "~하시오."의 명령조 형식으로 끝나야 한다.

예를 들어, 질문은
###
"글에 등장하는 화자들이 자신의 입장을 각각 어떻게 변호할지 논하시오."
"제시문의 첫 단락과 마지막 단락에서의 주장의 차이점과 공통점을 설명하시오."
"글에서 등장하는 화자들의 주장의 차이점과 공통점을 설명하시오."
###
등의 형식이어야 한다.

생성 형식은
{format}
으로 해야한다.

새로운 제시문
###
{new_presentation[presentation_number]}
###
                '''

In [10]:
!huggingface-cli download MLP-KTLim/llama-3-Korean-Bllossom-8B-gguf-Q4_K_M --local-dir='./'

model_id = 'MLP-KTLim/llama-3-Korean-Bllossom-8B-gguf-Q4_K_M'

tokenizer = AutoTokenizer.from_pretrained(model_id, verbose = 0)
model = Llama(
    model_path='/content/llama-3-Korean-Bllossom-8B-Q4_K_M.gguf', #다운로드받은 모델의 위치
    n_ctx=5000,
    n_gpu_layers=-1, verbose = 0        # Number of model layers to offload to GPU
)

messages = [
    {"role": "system", "content": f"{PROMPT}"},
    {"role": "user", "content": f"{test_text}"}
    ]

prompt = tokenizer.apply_chat_template(
    messages,
    tokenize = False,
    add_generation_prompt=True, verbose = 0
)

generation_kwargs = {
    "max_tokens":2000,
    "stop":["<|eot_id|>"],
    "top_p":0.95,
    "temperature":0.5,
    "echo":True, # Echo the prompt in the output
}

resonse_msg = model(prompt, **generation_kwargs)
generated = resonse_msg['choices'][0]['text'][len(prompt):]
print(generated)
print(f'질문 개수 : {num_of_question}')
qa_list.append(generated)
df_list.append([num_of_question, new_presentation[presentation_number], PROMPT, test_text, generated])

Fetching 6 files: 100% 6/6 [00:00<00:00, 4172.05it/s]
/content


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


질문 1:
제시문에서 사용된 유대교 신앙의 영향에 대한 논의는 무엇일까요?

답변 1:
제시전에서 다루는 내용에서는 유대교 신앙이 독일인들이 반유대적 동기를 갖게 하는 주요 요소로 제시되고 있습니다. 유대교 신앙은 그들의 생활과 신념에 있어서 중요한 역할을 하며, 반유대주의 성장 및 유지에 영향을 끼쳤을 것으로 추측됩니다.

질문 2:
제시문에서 나타나는 다른 학문의 영향은 무엇일까요?

답변 2:
제시전에서는 역사학자와 철학자 등 다른 학문의 이론이 언급되어 있습니다. 예를 들어, 제시에서 역사학자들은 과거에 일어난 사실을 재구성하여 지식의 발전에 기여하는 역할을 수행한다는 내용으로 논의됩니다. 또한 철학자들의 생각이나 유대교 신앙에 대한 이해도 제시되고 있습니다.

질문 3:
제시문에서 나타나는 이야기는 어떤 것인지 설명해주세요.

답변 3:
제시에서는 여러 가지 이야기가 다루어지고 있습니다. 예를 들어, 유대인 학살 명령의 실행과 그 결정적 원인들에 대한 이야기가 다루어지고 있습니다. 또한 101예비경찰대의 재판 기록에서 등장하는 이야기도 포함되어 있으며, 이를 통해 독일 사람들의 행동과 사고 방식이 설명되고 있습니다. 이러한 이야기는 제시에 포함되며, 독자에게 다양한 관점을 통해 이해될 수 있게 됩니다.
질문 개수 : 3


In [11]:
import re



# 정규 표현식을 사용하여 질문 부분만 추출합니다.
questions = re.findall(r'질문 \d+:\n(.*?)\n', generated, re.DOTALL)

# 각각의 질문을 다른 변수에 저장합니다.
question1, question2, question3 = [q.strip() for q in questions]

# 각 변수의 내용을 출력합니다.
print("Question 1:", question1)
print("Question 2:", question2)
print("Question 3:", question3)


Question 1: 제시문에서 사용된 유대교 신앙의 영향에 대한 논의는 무엇일까요?
Question 2: 제시문에서 나타나는 다른 학문의 영향은 무엇일까요?
Question 3: 제시문에서 나타나는 이야기는 어떤 것인지 설명해주세요.


In [12]:
df = pd.DataFrame(df_list, columns=columns)
df

,질문 개수,제시문,프롬프트,instruction,생성 질문
0,3,\n진실을 추구하지만 이야기라는 틀을 벗어날 수 없는 혼종 학문인 역사학은 인문학의...,\n당신은 대한민국에서 대입 논술 면접 학원을 운영하고 있는 선생님입니다.당신의 주...,"다음 제시문을 보고, 질문과 답변 세트 3개 생성하라. 다른 불필요한 문장은 생성하...",질문 1:\n제시문에서 사용된 유대교 신앙의 영향에 대한 논의는 무엇일까요?\n\n...


In [13]:
!pip install evaluate

In [18]:
model_id = 'MLP-KTLim/llama-3-Korean-Bllossom-8B-gguf-Q4_K_M'
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = Llama(
    model_path='./llama-3-Korean-Bllossom-8B-Q4_K_M.gguf', #다운로드받은 모델의 위치
    n_ctx=10000,
    n_gpu_layers=-1        # Number of model layers to offload to GPU
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
llama_model_loader: loaded meta data with 22 key-value pairs and 291 tensors from ./llama-3-Korean-Bllossom-8B-Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = .
llama_model_loader: - kv   2:                           llama.vocab_size u32              = 145088
llama_model_loader: - kv   3:                       llama.c

In [30]:
PROMPT = \
'''
You are a teacher who runs a college entrance essay interview academy in South Korea. Your main job is to generate new questions from the presentation.
When a new presentation is given, you identify the contents of the presentation and generate questions and answers related to it.
In addition, questions should be compared to each other or asked for relevance, taking into account the relationship between each presentation.
It should be a question about the connection between each presentation.
The questions should be specific to the whole presentation, not just specific content.
'''

instruction_1 = '''First, I'll show you an example of set of prompts. This set consists of four prompts.

[제시문 (가)]
예술은 새롭게 느끼고 생각하게 함으로써 인간과 현실과 세계를 신선하게 경험하도록 한다. 예술의 언어는
인간을 폭넓게 이해하며, 현실의 숨은 모습을 드러내고, 세계의 깊이를 자각하게 한다. 기존의 것을 있는 그
대로 받아들이지 않고 새롭게 이해하는 것은 대상의 변형 가능성을 찾아내는 것이다. 이러한 변형 가능성은
새 세계를 개시(開示)할 수 있는데, 바로 이 점이 보편성에 대한 예술의 대응 방식이 철학, 사회과학 혹은
자연과학과는 다르다는 것을 보여준다. 예술은 처음부터 보편성이나 전체성을 말하지 않는다. 그렇듯이 ‘미
(美)’ 혹은 ‘인간성’을 직접 말하지 않는다. 프로파간다 예술1)이나 사회주의 리얼리즘2)에서는 이를 직접 말
하기도 하지만, 이런 종류의 작품이 갖는 한계는 분명하다. 옳고 선하고 아름다운 일과 좋은 이념은 실현되
기 어렵기 때문에 버거운 것이다. 자유와 정의, 평등과 인권 같은 보편적인 가치들은 시종일관 반성되지 않
으면 거짓이기 쉽다. 검토와 비판을 허락하지 않는 보편성은 가짜이다. 참된 보편성은 개별적 경험과 특수한
사례의 지속적 검토를 견뎌낼 수 있어야 한다. 처음부터 보편타당한 원리를 상정하는 것이 아니라, 이 원리
의 갱신 가능성을 구체적 경험 속에서 늘 다시 검토해야 한다. 보편성의 검토 작업에서 예술의 위치, 역할,
메커니즘은 독특하다. 그 이유는 첫째, 예술은 개념적인 추상이 아니라 구체적인 경험에 천착하고, 둘째, 이
를 보편성의 지평 안으로 유도하여 재해석하는 활동이기 때문이다. 예술은 구체적 보편성 혹은 개별적 일반
성의 차원으로 나아간다. 특히 감각적인 경험과 구체적인 현실에 대한 밀착이야말로 예술의 제1덕목이다. 이
러한 밀착을 통해 예술은 흩어지고 무너지고 짓밟히고 외면된 것을 새롭게 읽어내고자 한다. 예술의 저항은
선언의 낭독이나 거리에서의 궐기 같은 직접적인 방식이 아니며, 설명이나 진단 혹은 분석과 같은 사회과학
적 방식 또한 아니다.
그러나 분명한 것은 예술이 단순히 운명의 필연성에 복속되지 않고 그 나름으로 대응하고 있다는 점이다.
예술은 드러나지 않을 수 있지만 미세하고도 오랜 파문을 일으킨다는 점에서 가장 강력한 저항매체라고 할
수 있다. 예술은 실천적이지만 행동주의적인 것이 아니며, 성찰적이지만 사변적인 것은 아니다. 심미적 개입
은 사유와 실천 사이에서 감각적인 경험과 구체적인 현실에 밀착하는 가운데 부단히 이루어진다. 그런 점에
서 예술은 운명의 악순환 고리를 끊으면서 우리를 더 넓고 깊은 행복으로 이끌고 나아간다고 말할 수 있다.
예술작품을 놓고 그것이 뜻하는 바에 대한 진위나 가치를 따질 수는 없다. 그런 방식으로 예술작품을 대한
다면, 그것은 이미 예술작품이 아닌 것이다. 시나 소설과 같은 예술작품은 일상적으로 사용되는 언어와 달
리, 객관적 사실을 표상하거나 구체적인 감정이나 생각을 표현하는 것도 아니고, 행위를 유도하기 위해 쓰이
지도 않는다. 따라서 한 예술작품에 대해 말할 때, 예술가의 의도나 생각과 같은 비언어적 맥락은 예술작품
의 의미를 결정하는 데 도움이 되지 않는다. 한마디로 말해 예술작품의 의미는 예술가의 의도일 수 없다.

[제시문 (나)]
예술작품이란 인간의 가치를 표현하고 형상화한 하나의 형식이다. 이 추상화된 형식은 관객에게 즐거움을
줄 수 있지만, 형식을 단순히 추상적 도안으로 해석해서는 안 된다. 형식은 표상적이고 재현적이며 함축적인
요소의 결합체이다. 형식을 이렇게 해석할 때 예술에 적용되는 의미가 풍부해지고 인간의 가치 위계에서 당
연히 높은 위치를 점하게 된다. 많은 예술가가 자신을 위해 작품을 만든다고 하지만, 사실은 자신의 규범을
설정하고 있는 것이다. 예술작품은 사회적인 반응과 수렴을 요구하는 관계 속에서 비로소 의미를 가진다. 예
술가는 예술 경향의 여러 갈래 속에서 자신만의 고유한 작업을 통해 인성을 창조하며, 인간적 고뇌와 고통
을 풀어보려 노력한다. 이를 통해 예술가는 현세의 이상향을 찾아 나서기도 한다. 실제로 모든 종류의 가치,
이를테면 도덕적, 종교적, 경제적, 과학적 가치 등은 적절한 환경 아래에서 미적으로 관조된다. 이러한 기초
위에서 예술이 인간의 모든 가치 영역을 표현하거나 포함한다고 볼 수 있다. 예술이 직접적인 혹은 특별한
의무를 명시하지는 않더라도, 그 영향력으로 우리가 언제나 고귀한 존재로 남을 수 있게 해준다.
예술은 인간을 감성적, 지적, 도덕적으로 성장시킨다는 측면에서 교육적 기능을 담당한다고 볼 수 있다. 시
나 소설과 같은 문자 예술은 독자에게 과학으로는 결코 도달할 수 없는 인간에 대한 진실성을 보여주고, 사
회나 역사에 대한 의식과 지식의 폭을 넓혀주며, 도덕적 감수성을 고양하는 데 크게 기여한다. 위대한 소설
을 읽었을 때, 훌륭한 연극을 보았을 때, 우리는 흔히 커다란 정신적 충격을 경험한다. 도스토옙스키의 「죄
와 벌」을 읽거나 셰익스피어의 「리어왕」을 관람한 후에 우리는 그동안 알아왔던 세계가 뒤바뀌고 뒤집히
는 느낌을 가질 수 있다. 이런 작품들은 우리가 지금 살고 있는 세계 속에 안주하는 것을 용납하지 않는다.
또한 세계를 새롭게 볼 수 있게 하고, 새로운 감각으로 우리 자신을 느끼게 하며, 행위의 의미를 새로운 시
각에서 조명하게 한다. 예술을 통한 이러한 경험은 비단 문학작품에서만 나타나는 것은 아니다. 위대한 그림
을 보았을 때, 멋진 무용을 구경했을 때, 훌륭한 음악을 들었을 때도 마찬가지이다. 예술작품과의 접촉을 통
해 우리는 사물 현상을 다른 차원에서 신선하게 느끼며, 우리의 행위를 새로운 도덕적 척도에서 평가하게
되는 것이다. 이를 통해 우리는 세계와 우리 자신이 달라진 것을 발견할 수 있다.

[제시문 (다)]
음악은 인간의 마음을 선하게 한다. 평화로운 음악은 사람으로 하여금 화목하고 상호 공경하게 하여 방탕
에 빠지지 않도록 하며, 엄숙하고 장엄한 음악은 마음을 가지런하게 하여 어지럽지 않게 한다. 그러므로 마
음을 감동시키고 풍속을 바꾸는 데에는 음악보다 더 좋은 것이 없다. 음악은 사람 각자의 심성을 교화시키
고, 나아가 사회 구성원 간의 조화로움을 지향한다. 이를 유가(儒家)에서는 예교(禮敎)와 악교(樂敎)를 통합
하여 ‘예악교화(禮樂敎化)’라고 이른다. 예악(禮樂)은 사람들이 도덕적 선(善)을 성취하는 도덕 수양에서 중
요한 수단으로 간주된다. 여기서 예(禮)는 이지(理智)의 측면에서 말한 것이고, 악(樂)은 감정의 측면에서
말한 것이다. 이지는 차이를 구별하는 것이고, 감정은 화합하는 것이다. 그러므로 예의는 인륜 관계를 구별
하여 각자 자신의 위치를 편안히 여기고 그 직분을 담당하게 하며, 음악은 집단의 감정을 조화롭게 하고 의
사소통을 통하여 마음을 융합하게 만든다. 어느 학자는 “음악은 인간의 감정을 화합하며, 예는 인륜 관계의
차이를 구별한다”고 지적했는데, 이것은 예악의 상대적 역할과 조화를 말한 것이다. 또 『예기(禮記)』에서
는 “음악은 안에서부터 나오며, 예는 밖에서부터 생겨난 것이다”라고 하였다. 이처럼 예와 악이 하나는 안에
서, 하나는 밖에서 상호 조절하여 이지와 감정 모두 평정을 얻게 된다. “음악이 잘 통하면 원망이 없고 예가
잘 통하면 다툼이 없으니, 서로 양보하여 천하를 다스리는 것은 예악을 두고 한 말이다”라는 구문에서 이를
확인할 수 있다. “예악을 통합하여 인간의 마음을 다스린다”는 것은 예악의 효용을 의미한다. ‘마음을 다하여
변화를 극진하게 하는 것’은 음악이 마음 깊숙한 곳을 울려 감정의 변화를 극진히 하는 것을 뜻하며, ‘성실
함을 밝히고 허위를 제거하는 것’은 예가 공경의 마음을 표현하고 거짓 행위를 제거하는 것을 말한다. 그래
서 “음악이 행해지면 뜻이 맑아지고, 예를 닦으면 덕행이 이루어진다”, “예에서 사람이 서고, 악에서 사람이
완성된다”는 등의 말은 예술과 도덕의 조화로운 관계 속에서 도덕적으로 완성된 인격체로서 거듭날 수 있음
을 강조한 것이다. 이와 더불어, “음악이 나쁜 풍속을 좋은 방향으로 변화시키니 천하가 모두 태평해진다”라
는 말은 예술이 결코 인간의 공동체적 삶과 분리될 수 없으며, 구성원의 삶을 인격적으로 고양하고 공동체
의 안녕과 발전에 기여해야 한다는 것이다. 결국 ‘예술을 위한 예술’은 존재할 수 없으며, 개인적 차원이나
공동체 차원에서 선을 추구하고 이를 지향하는 경우에만 진정한 예술이 된다.

[제시문 (라)]
예술은 의미의 문제가 아니라 감각적인 질과 구상(構想)의 문제이다. 예술의 재료가 의미중립적인 것이든
인간적인 것이든, 우리는 예술의 재료와 이 재료로 만들어진 예술작품을 구분해야 한다. 예술작품의 창조나
감상에 대한 관심은 재료 자체에 있는 것이 아니라 재료로 만들어진 결과물에 있다. 그러나 그 모두가 바로
예술작품이 될 수는 없으며, 사람들이 흥미나 감흥을 느낄 때에만 비로소 ‘미적’이라고 할 수 있다. 예술의
관심이 사회의 규칙과 원리에 포함된다는 시각이나 이러한 원리를 예술에 적용하려는 시도는 미적 창조와
감상 모두에 유해한 것이다. 미적 대상은 인식적인 가치로부터 분리되어 있으며, 감각적이거나 형식적인 성
질 자체를 깨달았을 때의 순수한 기쁨에서 정당성을 얻는다. 아름다운 사물에서 ‘아름다움’의 의미를 찾아내
고 받아들이는 것은 교양 있고 선택된 사람의 특권이다. 셰익스피어의 작품은 운문(韻文)의 수려함, 비유적
표현의 다양함과 풍부함, 그리고 구성의 탁월함 때문에 상찬을 받는다. 예술가의 창조성은 그 자체로 목적이 된다.
그렇다면 예술은 무엇에 도움을 줄 수 있는가? 상업주의는 도구적 문화를 요구하며 예술의 유용성만을 강
조하기 때문에 비판에 직면하고 있다. 음미되는 가치 자체만으로 예술은 존재의 이유가 있다. 예술은 절대로
어떤 것에도 도움을 주지 않으며, 역사, 사회, 그리고 개인의 맥락과는 단절된 자율적 유기체이다. 계산될 수
있는 한에서 세계는 과학의 지배를 받기 마련이지만, 예술의 왕국은 ‘계산될 수 없는 곳’에서 시작된다. 예술
은 매 순간 자기 충만성을 향유하게 하며, 경험을 생기 있게 하고, 경험이 지닌 상상적이고 정서적인 기쁨을
즐기게 한다. 좋은 삶이란 경험의 매 순간을 풍부하고 정열적으로 사는 것이다. 그러므로 예술을 사랑하는
것이야 말로 좋은 삶의 원천이 된다.

Next, I will show you the example question. This question is generated from the sample set of prompts above.

###
제시문(가) ∼ 제시문(라)는 예술에 관한 견해를 담고 있다. 제시문들을 상반된 두 입장으로 분류하고 각 입장을 요약하시오.
###

In this way, the example question references the content of the sample prompts, summarizing them or asking about topics that can be considered from them. When creating question, it's important to consider the relationships between the contents of each prompt.
Now I'll give you a new prompt. You have to generate question from new prompt, and It should be created in the same format and tone as this example question.
Do you understand? Just answer Yes or no.
'''

instruction_2 = '''
Please create one question and one answer based on the following prompts. You need to generate questions from the [new prompts]. When generating question, the content should not imply or suggest the answers. Please delete any unnecessary sentences.

Please create one question and one answer. As I said, when generating question, the content should not imply or suggest the answer. When creating question, it's important to consider the relationships between the contents of each prompt. The question should be in Korean, and concise, consisting of 1-2 sentences.


[new prompts (가)]
협력이 가능한 일반적인 요건은 두 가지이다. 첫째, 협력은 호혜주의를 바탕으로 한다. 둘째, 호혜주의가 오
랜 기간 동안 안정적으로 유지될 수 있다는 믿음이 있어야 한다. 결국 서로가 상대방을 도울 수 있을 때 협
력이 가능하다. 그런데 도울 때 대가를 치러야 한다면 딜레마가 생길 수 있다. 협력 당사자의 입장에서 보면
상대방의 협력 덕택에 얻는 이득이 내가 협력할 때 치러야 할 비용보다 더 커야만 협력할 이유가 생겨서 상
호 이득의 기회를 현실화할 수 있다. 이 경우 협력 당사자 모두가 상호 협력을 상호 배반보다 더 선호하게
되는 것은 당연하다. 그러나 내가 원하는 것을 얻기가 쉽지는 않다. 다음 두 이유 때문이다. 첫째, 나는 도움
을 주지 않는 것이 단기적으로 더 이득이지만 그럼에도 불구하고 상대방은 나를 돕도록 유도하고 싶어 한
다. 둘째, 나는 남에게 큰 비용이 드는 도움을 제공하지 않으면서도 받을 수 있는 도움은 모두 받고 싶은 유
혹을 느낀다. 하지만 일단 한 집단 안에서 호혜주의를 바탕으로 협력이 자리를 잡으면 어떤 비협력적인 전
략도 침범하기 어려워서 스스로를 지켜낼 수 있다.
협력이 배태될 수 있고, 온갖 다양한 전략이 뒤섞여 있는 환경에서도 번성할 수 있으며, 또 일단 자리를 잡
은 뒤에는 스스로를 보호할 수 있다는 사실은 매우 고무적이다. 하지만 무엇보다 흥미로운 사실은 이러한
결과를 얻는데 개인이나 사회적인 환경의 특성은 거의 영향을 미치지 못한다는 점이다. 개인은 논리적일 필
요가 없고 어떻게 하고 왜 그렇게 해야 하는지 알지 못해도 된다. 진화 과정은 성공적인 전략들이 자연적으
로 번성하게 해 준다. 또한 개인은 서로 메시지나 약속을 주고받을 필요도 없다. 말도 필요하지 않다. 행동
이 말을 대신하기 때문이다. 마찬가지로 개인 사이에 어떤 신뢰도 필요하지 않다. 호혜주의만으로 충분히 배
반을 비생산적이 되게 할 수 있기 때문이다. 이타주의도 필요하지 않다. 성공적인 전략은 이기주의자한테서
도 협력을 이끌어내기 때문이다.
협력의 창발과 성장, 유지에 꼭 필요한 개인과 사회 환경에 대한 가정이 몇 가지 있기는 하다. 우선 예전에
상호 작용했던 협력자를 알아볼 수 있어야 하며 상대와의 과거 상호 작용 내력을 기억할 수 있어야 한다. 그
래야 그에 따라 반응할 수 있다. 하지만 실제로는 인식과 기억의 요건을 맞추는 것이 그렇게 어려운 것은 아
니다. 박테리아조차도 단 하나의 상대하고만 상호 작용한다거나, 단 하나의 전략을 가지고 상대방의 가장 최
근 행위에 대해서만 반응하는 식으로 이런 요건을 충족시킨다. 박테리아가 할 수 있으면 사람도 할 수 있다.

[new prompts (나)]
인간의 노력을 조정하는 수단으로 경쟁의 힘을 최대한 잘 활용해야 한다. 이는 유효한 경쟁이 창출될 수
있는 곳에서는 그 어떤 방법보다도 이것이 개별적 노력의 좋은 길잡이가 된다는 확신에 기초한 것이다. 나
는 경쟁이 유익하게 작동하려면 세심하게 배려된 법적 틀이 필요하다는 사실을 부인하지 않는다. 또한 만약
경쟁이 유효하게 이루어지도록 하는 조건을 충족시키기 어려운 경우에는 다른 방법을 경제활동의 길잡이로
삼아야 한다는 것도 부정하지 않는다. 그러나 개인의 개별적 노력을 조정하는 방법으로 더욱 열등한 방법이
경쟁을 대체하는 것에는 반대한다.
그리고 알려진 방법 중 가장 효율적이라는 이유뿐만 아니라 더 크게는 권력의 강제적이고도 자의적인 간섭
없이도 우리의 행위를 조정할 수 있는 유일한 방법이기 때문에 나는 경쟁을 우월한 방법으로 간주한다. 사
실, 경쟁을 선호하는 핵심 이유 중 하나는 의식적인 사회적 통제가 필요하지 않다는 점이며, 특정한 직업이
그 직업과 연관된 불리한 점과 위험 요소를 상쇄하고도 남을 만큼 전망이 있는지 스스로 판가름할 기회를
각자에게 부여한다는 점이다. 경쟁을 사회 조직의 원칙으로 잘 활용하기 위해서는 강제적 간섭을 배제해야
하지만 그 작동을 도와줄 수 있는 특정한 방식의 간섭은 인정할 수 있으며 정부의 적절한 조치가 필요한 경
우도 있다. 그러나 강제력을 사용해서는 안 된다는 점은 특별히 강조해야 한다. 시장 참여자는 거래 상대방
을 찾을 수 있는 한 어떤 가격에서건 자유롭게 팔고 살 수 있어야 하고, 누구든 자유롭게 생산할 수 있고,
팔릴 수 있는 어떤 것도 생산하여 팔고 살 수 있어야 한다.
업종에 상관없이 모든 사람에게 같은 조건으로 진입을 자유롭게 허용해야 한다는 것은 본질적으로 중요하
다. 아울러 개인이나 단체가 공개적인 혹은 드러나지 않은 힘을 이용하여 이러한 진입을 제한하려는 것을
법이 용인하지 않아야 한다는 것 또한 본질적으로 중요하다. 특정 상품에 대해 가격이나 물량을 통제하게
되면 각자의 노력을 유효하게 조정하는 경쟁의 능력은 박탈된다. 왜냐하면 이 경우 가격의 변화가 변화된
모든 상황을 더 이상 반영하지 못하게 되며, 그 결과 가격의 변화가 더 이상 개인의 행위를 나타내는 믿을
만한 길잡이가 될 수 없기 때문이다.

[new prompts (다)]
전체 사회 구성원이 공동으로 소유한 산림과 바다 같은 공유 자원은 배제성은 없지만 경합성이 있다. 배제
성이 없으므로 대가를 지불하지 않고도 누구나 공유 자원을 소비할 수 있는 반면, 경합성이 있으므로 누군
가가 공유 자원을 소비하면 다른 사람의 소비 기회는 감소한다. 따라서 우리는 공유 자원을 다른 사람보다
먼저 더 많이 소비하려고 경쟁한다. 이와 같은 특성 때문에 공유 자원은 쉽게 남용되어 필요한 양보다 과다
하게 소비하게 되고 고갈될 가능성이 커진다. 이에 대해 일부 공동체주의자는 공동체적 해법이야말로 인간
의 본성에 가장 걸맞은 자연스러운 해법이라고 주장한다. 이들에게 자연은 “적자생존” 혹은 “피로 물든 이빨
과 발톱” 등으로 묘사되어온 생존 경쟁의 장이 아니다. 오히려 이들은 상호 부조의 감정이야말로 수십만 년
에 걸친 집단생활을 통해 그리고 지난 수천 년의 사회생활을 통해 배양된 것이며, “전쟁터에서처럼 사람들이
미쳐 돌아가는 상태가 아니라면” 상호 부조의 감정은 거스를 수 없는 법이라고 주장한다.
그러나 인간은 언제나 공공의 이득을 위해서 행동할 준비가 되어 있는 존재는 아니다. 그렇기 때문에 인간
본성에 기댈 수만은 없으며 적절한 유인이 필요하고 적절한 제도가 필요하다. 적절한 제도 하에서는 자신의
이익과 공동의 이익을 조화시킬 수 있을 것이기 때문이다. 이때 적절한 제도란 물질적 이득을 추구하는 개
인을 외적으로 그리고 위에서 아래로 강제하는 장치가 아니라 상호 신뢰를 통해 서로를 규제할 수 있는 견
제 장치이다. 이것이 가능한 것은 인간은 협조하려는 상대방의 의지가 확인되면 언제든지 이에 협조로 응답
하고, 자신의 행동이 타인에게 미치는 영향을 고려하며, 다른 누군가의 행동이 타인에게 해를 입히는 경우
자기 일처럼 나서서 이를 제어하려는 의향을 가진 존재이기 때문이다.
1960년대 오스트레일리아에서 바다가재를 잡는 지역의 어부들은 바다가재의 숫자가 계속 줄어드는 문제에
직면해 있었다. 문제를 해결하기 위해 어부들은 좋은 생각을 해냈다. 공동체를 결성해서 설치할 수 있는 어
망의 숫자를 제한하기로 한 것이다. 이에 다른 사람이 어망을 더 설치하는 것을 서로 감시하게 되면서 불법
적으로 어망을 설치하는 어민이 사라졌고 결국 바다가재의 숫자를 늘리는 데 성공했다. 만약 정부에서 일방
적으로 어망을 제한했다면 여러 나라에서 그랬던 것처럼 일부 어부가 경비정 몰래 바다가재를 제한 없이 잡
다가 경비정이 쫓아오면 잡히지 않기 위해 그물을 끊고 도망가는 불법 조업이 기승을 부렸을 것이다. 그런
데 어부들이 자율적으로 제한을 두면서 자연스럽게 불법 조업이 자취를 감추었다.
공동체를 토대로 사회질서를 유지할 수 있다는 생각을 누군가는 낡은 이념으로, 다른 누군가는 유토피아적
이념으로 여긴다. 그러나 많은 경우 공동체 구성원이 상호 감시하고 상호 제재하는 것이 공유 자원을 관리
하는 효과적인 방법이 된다. 공동체 구성원은 외적 권위체가 갖지 못하는 정보를 갖고 있으며, 공동체 내에
서 상호 신뢰를 기초로 서로를 규제해가면서 비극을 피할 수 있기 때문이다.

[new prompts (라)]
경쟁을 통해 각자의 능력이 최대한 발휘되기도 하지만 경쟁에서 진 쪽은 여러 가지 부작용을 겪기도 하는
것이 사실이다. 경쟁이란 본질적으로 ‘차가운 것’이기 때문이다. 그래서 시장에서는 경쟁에서 지지 않기 위해
경쟁관계의 두 기업이 일시적으로 협력하거나 다양한 방식의 협력관계를 경쟁 속에 부분적으로 도입하는 역
설적인 경우를 볼 수 있다. 예를 들어 오랫동안 경쟁하다가도 경쟁자에게 부품을 공급하는 협력적 공급자로
위치가 일시적으로 변화할 수 있다. 협력을 도입하여 경쟁의 형태가 간접적인 것으로 변화할 수도 있다. 예
들 들어 한 기업이 다른 두 기업과 각각 독립적으로 파트너십을 맺고 협력하여 양쪽의 파트너 기업들이 서
로 경쟁하는 구도가 만들어 질 수도 있다. 판매에서는 경쟁하지만 연구 및 개발에서는 잠정적으로 협력한다거
나 아예 부서를 나누어서 어떤 부서에서는 협력을, 다른 부서에서는 경쟁을 진행하는 방식으로 부분적 협력이
진행될 수도 있다. 특히 새로운 기술이 등장해서 기술 혹은 산업간 융합이 일어날 때, 그리고 기업 규모가
작아서 경쟁력을 높여야 할 때 경쟁자와 협력하는 현상이 일어나기도 한다. 그럼에도 경쟁 메커니즘은 근본
적으로 기업들이 경쟁에서 우위를 확보하기 위해 혁신에 더 집중하도록 유도한다. 일반적으로 기업들은 독
점보다는 경쟁 상황일 때 연구개발 등 혁신 활동에 더 많이 투자할 유인을 갖기 때문이다.
사회 전체 차원에서 보면 경쟁이 심해질수록 사회생활의 접촉대상이 되는 타인은 실재적 혹은 잠재적 경쟁
대상으로만 간주될 것이며 이러한 사고가 팽배할 때 휴머니즘을 토대로 한 사회적 친화력을 견지하기는 어
렵다. 그래서 경제사상가들은 경쟁이 국부의 증대에 기여하지만 사회적 동질성과 도덕을 잠식한다는 지적을
자주 해 왔다. 이런 문제를 고려할 때 제기되는 과제는 한편으로는 경쟁의 기능을 계속 살리면서 다른 한편
으로는 경쟁이 유발하는 비인도적인 측면을 약화시킬 수 있는 ‘황금의 중도’를 설정하는 일이다. 2차 세계대
전 이후 독일을 위시한 몇몇 서구국가들이 도입한 사회적 시장경제 제도는 바로 그와 같은 사실을 배경으로
형성된 것이었다. 효율성을 증대시키는 시장경제의 기능을 토대로 하면서 동시에 자본주의 사회에 내재하는
차가움이나 사회·경제적 역기능을 극소화하자는 것이다.
최근 들어 과거의 성장제일주의 정책을 수정하여 안정과 형평 내지는 삶의 질을 조화시키겠다고 선언하는
국가를 쉽게 찾을 수 있다. 이를 위해서는 무엇보다도 사회의 기본적 가치에 대한 공감이 형성되지 않으면
안 된다. 그렇지 않을 경우 표출되는 다양한 갈등은 사회적 제어능력을 벗어날 위험이 있기 때문이다. 공존
원리로서의 공정한 경쟁을 통한 진보를 훼손하지 않으면서도 휴머니즘을 기저로 하는 사회적 삶의 실존과
조화되는 규범을 마련하는 토론의 장에 모든 사회 구성원 혹은 이익집단의 참여가 필요한 이유도 여기에 있
다. 최근 폭발적으로 분출되는 다양한 욕구·견해·주장 등은 아마도 새로운 질서창출을 위한 사회적 공감대
형성의 불가피한 진통일 것이다.

'''

messages = [
    {"role": "system", "content": f"{PROMPT}"},
    {"role": "user", "content": f"{instruction_1}"}
    ]

prompt = tokenizer.apply_chat_template(
    messages,
    tokenize = False,
    add_generation_prompt=True
)

generation_kwargs = {
    "max_tokens":512,
    "stop":["<|eot_id|>"],
    "top_p":0.9,
    "temperature":0.2,
    "echo":True, # Echo the prompt in the output
}

resonse_msg = model(prompt, **generation_kwargs)
print(resonse_msg['choices'][0]['text'][len(prompt):])

Llama.generate: prefix-match hit

llama_print_timings:        load time =     141.95 ms
llama_print_timings:      sample time =       7.57 ms /     3 runs   (    2.52 ms per token,   396.35 tokens per second)
llama_print_timings: prompt eval time =    1781.73 ms /  2475 tokens (    0.72 ms per token,  1389.10 tokens per second)
llama_print_timings:        eval time =      55.70 ms /     2 runs   (   27.85 ms per token,    35.91 tokens per second)
llama_print_timings:       total time =    1854.52 ms /  2477 tokens


Yes.


In [31]:
messages_2 = [
    {"role": "system", "content": f"{PROMPT}"},
    {"role": "user", "content": f"{instruction_2}"}
    ]

prompt_2 = tokenizer.apply_chat_template(
    messages_2,
    tokenize = False,
    add_generation_prompt=True
)

generation_kwargs = {
    "max_tokens":512,
    "stop":["<|eot_id|>"],
    "top_p":0.9,
    "temperature":0.2,
    "echo":True, # Echo the prompt in the output
}

resonse_msg = model(prompt_2, **generation_kwargs)
print(resonse_msg['choices'][0]['text'][len(prompt_2):])

Llama.generate: prefix-match hit

llama_print_timings:        load time =     141.95 ms
llama_print_timings:      sample time =     376.00 ms /   155 runs   (    2.43 ms per token,   412.24 tokens per second)
llama_print_timings: prompt eval time =    1713.35 ms /  2389 tokens (    0.72 ms per token,  1394.34 tokens per second)
llama_print_timings:        eval time =    3805.43 ms /   154 runs   (   24.71 ms per token,    40.47 tokens per second)
llama_print_timings:       total time =    6090.06 ms /  2543 tokens


Here is the question and answer based on the new prompts:
  
  [질문] 경쟁과 협력이 필요한 때, 어떤 방식을 선택할까요?
  
  [답변] 경쟁과 협력은 모두 중요한 전략이지만, 서로 다른 목적과 결과를 갖습니다. 경쟁은 각 개인이나 조직이 생존 및 성공하기 위해 다른 개인이나 조직과의 차별화를 추구하며, 이는 주로 혁신, 마케팅, 및 경영 등의 분야에서 이루어집니다. 반면에 협력은 서로 상호 작용하는 개인이나 단체가 함께 목표를 달성하기 위해 협력하는 것을 의미합니다. 협력은 호혜주의, 공동 이익 등을 바탕으로 하며, 이러한 협력이 지속 가능한 경쟁 환경을 조성하는데 도움이 됩니다. 따라서 어떤 상황에서 경쟁과 협력을 선택할지는 해당 상황의 목적과 목표에 따라 결정되어야 합니다.


In [23]:
PROMPT = \
'''
Here is a sample set of prompts given as an example. This set consists of four prompts.

[제시문 (가)]
예술은 새롭게 느끼고 생각하게 함으로써 인간과 현실과 세계를 신선하게 경험하도록 한다. 예술의 언어는 인간을 폭넓게 이해하며, 현실의 숨은 모습을 드러내고, 세계의 깊이를 자각하게 한다. 기존의 것을 있는 그대로 받아들이지 않고 새롭게 이해하는 것은 대상의 변형 가능성을 찾아내는 것이다. 이러한 변형 가능성은 새 세계를 개시(開示)할 수 있는데, 바로 이 점이 보편성에 대한 예술의 대응 방식이 철학, 사회과학 혹은 자연과학과는 다르다는 것을 보여준다. 예술은 처음부터 보편성이나 전체성을 말하지 않는다. 그렇듯이 ‘미
(美)’ 혹은 ‘인간성’을 직접 말하지 않는다. 프로파간다 예술1)이나 사회주의 리얼리즘2)에서는 이를 직접 말하기도 하지만, 이런 종류의 작품이 갖는 한계는 분명하다. 옳고 선하고 아름다운 일과 좋은 이념은 실현되
기 어렵기 때문에 버거운 것이다. 자유와 정의, 평등과 인권 같은 보편적인 가치들은 시종일관 반성되지 않으면 거짓이기 쉽다. 검토와 비판을 허락하지 않는 보편성은 가짜이다. 참된 보편성은 개별적 경험과 특수한
사례의 지속적 검토를 견뎌낼 수 있어야 한다. 처음부터 보편타당한 원리를 상정하는 것이 아니라, 이 원리의 갱신 가능성을 구체적 경험 속에서 늘 다시 검토해야 한다. 보편성의 검토 작업에서 예술의 위치, 역할,
메커니즘은 독특하다. 그 이유는 첫째, 예술은 개념적인 추상이 아니라 구체적인 경험에 천착하고, 둘째, 이를 보편성의 지평 안으로 유도하여 재해석하는 활동이기 때문이다. 예술은 구체적 보편성 혹은 개별적 일반
성의 차원으로 나아간다. 특히 감각적인 경험과 구체적인 현실에 대한 밀착이야말로 예술의 제1덕목이다. 이러한 밀착을 통해 예술은 흩어지고 무너지고 짓밟히고 외면된 것을 새롭게 읽어내고자 한다. 예술의 저항은
선언의 낭독이나 거리에서의 궐기 같은 직접적인 방식이 아니며, 설명이나 진단 혹은 분석과 같은 사회과학적 방식 또한 아니다.
그러나 분명한 것은 예술이 단순히 운명의 필연성에 복속되지 않고 그 나름으로 대응하고 있다는 점이다.
예술은 드러나지 않을 수 있지만 미세하고도 오랜 파문을 일으킨다는 점에서 가장 강력한 저항매체라고 할 수 있다. 예술은 실천적이지만 행동주의적인 것이 아니며, 성찰적이지만 사변적인 것은 아니다. 심미적 개입
은 사유와 실천 사이에서 감각적인 경험과 구체적인 현실에 밀착하는 가운데 부단히 이루어진다. 그런 점에서 예술은 운명의 악순환 고리를 끊으면서 우리를 더 넓고 깊은 행복으로 이끌고 나아간다고 말할 수 있다.
예술작품을 놓고 그것이 뜻하는 바에 대한 진위나 가치를 따질 수는 없다. 그런 방식으로 예술작품을 대한다면, 그것은 이미 예술작품이 아닌 것이다. 시나 소설과 같은 예술작품은 일상적으로 사용되는 언어와 달
리, 객관적 사실을 표상하거나 구체적인 감정이나 생각을 표현하는 것도 아니고, 행위를 유도하기 위해 쓰이지도 않는다. 따라서 한 예술작품에 대해 말할 때, 예술가의 의도나 생각과 같은 비언어적 맥락은 예술작품
의 의미를 결정하는 데 도움이 되지 않는다. 한마디로 말해 예술작품의 의미는 예술가의 의도일 수 없다.

[제시문 (나)]
예술작품이란 인간의 가치를 표현하고 형상화한 하나의 형식이다. 이 추상화된 형식은 관객에게 즐거움을 줄 수 있지만, 형식을 단순히 추상적 도안으로 해석해서는 안 된다. 형식은 표상적이고 재현적이며 함축적인
요소의 결합체이다. 형식을 이렇게 해석할 때 예술에 적용되는 의미가 풍부해지고 인간의 가치 위계에서 당연히 높은 위치를 점하게 된다. 많은 예술가가 자신을 위해 작품을 만든다고 하지만, 사실은 자신의 규범을
설정하고 있는 것이다. 예술작품은 사회적인 반응과 수렴을 요구하는 관계 속에서 비로소 의미를 가진다. 예술가는 예술 경향의 여러 갈래 속에서 자신만의 고유한 작업을 통해 인성을 창조하며, 인간적 고뇌와 고통
을 풀어보려 노력한다. 이를 통해 예술가는 현세의 이상향을 찾아 나서기도 한다. 실제로 모든 종류의 가치, 이를테면 도덕적, 종교적, 경제적, 과학적 가치 등은 적절한 환경 아래에서 미적으로 관조된다. 이러한 기초
위에서 예술이 인간의 모든 가치 영역을 표현하거나 포함한다고 볼 수 있다. 예술이 직접적인 혹은 특별한 의무를 명시하지는 않더라도, 그 영향력으로 우리가 언제나 고귀한 존재로 남을 수 있게 해준다.
예술은 인간을 감성적, 지적, 도덕적으로 성장시킨다는 측면에서 교육적 기능을 담당한다고 볼 수 있다. 시나 소설과 같은 문자 예술은 독자에게 과학으로는 결코 도달할 수 없는 인간에 대한 진실성을 보여주고, 사
회나 역사에 대한 의식과 지식의 폭을 넓혀주며, 도덕적 감수성을 고양하는 데 크게 기여한다. 위대한 소설을 읽었을 때, 훌륭한 연극을 보았을 때, 우리는 흔히 커다란 정신적 충격을 경험한다. 도스토옙스키의 「죄
와 벌」을 읽거나 셰익스피어의 「리어왕」을 관람한 후에 우리는 그동안 알아왔던 세계가 뒤바뀌고 뒤집히는 느낌을 가질 수 있다. 이런 작품들은 우리가 지금 살고 있는 세계 속에 안주하는 것을 용납하지 않는다.
또한 세계를 새롭게 볼 수 있게 하고, 새로운 감각으로 우리 자신을 느끼게 하며, 행위의 의미를 새로운 시
각에서 조명하게 한다. 예술을 통한 이러한 경험은 비단 문학작품에서만 나타나는 것은 아니다. 위대한 그림
을 보았을 때, 멋진 무용을 구경했을 때, 훌륭한 음악을 들었을 때도 마찬가지이다. 예술작품과의 접촉을 통
해 우리는 사물 현상을 다른 차원에서 신선하게 느끼며, 우리의 행위를 새로운 도덕적 척도에서 평가하게
되는 것이다. 이를 통해 우리는 세계와 우리 자신이 달라진 것을 발견할 수 있다.

[제시문 (다)]
음악은 인간의 마음을 선하게 한다. 평화로운 음악은 사람으로 하여금 화목하고 상호 공경하게 하여 방탕
에 빠지지 않도록 하며, 엄숙하고 장엄한 음악은 마음을 가지런하게 하여 어지럽지 않게 한다. 그러므로 마
음을 감동시키고 풍속을 바꾸는 데에는 음악보다 더 좋은 것이 없다. 음악은 사람 각자의 심성을 교화시키
고, 나아가 사회 구성원 간의 조화로움을 지향한다. 이를 유가(儒家)에서는 예교(禮敎)와 악교(樂敎)를 통합
하여 ‘예악교화(禮樂敎化)’라고 이른다. 예악(禮樂)은 사람들이 도덕적 선(善)을 성취하는 도덕 수양에서 중
요한 수단으로 간주된다. 여기서 예(禮)는 이지(理智)의 측면에서 말한 것이고, 악(樂)은 감정의 측면에서
말한 것이다. 이지는 차이를 구별하는 것이고, 감정은 화합하는 것이다. 그러므로 예의는 인륜 관계를 구별
하여 각자 자신의 위치를 편안히 여기고 그 직분을 담당하게 하며, 음악은 집단의 감정을 조화롭게 하고 의
사소통을 통하여 마음을 융합하게 만든다. 어느 학자는 “음악은 인간의 감정을 화합하며, 예는 인륜 관계의
차이를 구별한다”고 지적했는데, 이것은 예악의 상대적 역할과 조화를 말한 것이다. 또 『예기(禮記)』에서
는 “음악은 안에서부터 나오며, 예는 밖에서부터 생겨난 것이다”라고 하였다. 이처럼 예와 악이 하나는 안에
서, 하나는 밖에서 상호 조절하여 이지와 감정 모두 평정을 얻게 된다. “음악이 잘 통하면 원망이 없고 예가
잘 통하면 다툼이 없으니, 서로 양보하여 천하를 다스리는 것은 예악을 두고 한 말이다”라는 구문에서 이를
확인할 수 있다. “예악을 통합하여 인간의 마음을 다스린다”는 것은 예악의 효용을 의미한다. ‘마음을 다하여
변화를 극진하게 하는 것’은 음악이 마음 깊숙한 곳을 울려 감정의 변화를 극진히 하는 것을 뜻하며, ‘성실
함을 밝히고 허위를 제거하는 것’은 예가 공경의 마음을 표현하고 거짓 행위를 제거하는 것을 말한다. 그래
서 “음악이 행해지면 뜻이 맑아지고, 예를 닦으면 덕행이 이루어진다”, “예에서 사람이 서고, 악에서 사람이
완성된다”는 등의 말은 예술과 도덕의 조화로운 관계 속에서 도덕적으로 완성된 인격체로서 거듭날 수 있음
을 강조한 것이다. 이와 더불어, “음악이 나쁜 풍속을 좋은 방향으로 변화시키니 천하가 모두 태평해진다”라
는 말은 예술이 결코 인간의 공동체적 삶과 분리될 수 없으며, 구성원의 삶을 인격적으로 고양하고 공동체
의 안녕과 발전에 기여해야 한다는 것이다. 결국 ‘예술을 위한 예술’은 존재할 수 없으며, 개인적 차원이나
공동체 차원에서 선을 추구하고 이를 지향하는 경우에만 진정한 예술이 된다.

[제시문 (라)]
예술은 의미의 문제가 아니라 감각적인 질과 구상(構想)의 문제이다. 예술의 재료가 의미중립적인 것이든
인간적인 것이든, 우리는 예술의 재료와 이 재료로 만들어진 예술작품을 구분해야 한다. 예술작품의 창조나
감상에 대한 관심은 재료 자체에 있는 것이 아니라 재료로 만들어진 결과물에 있다. 그러나 그 모두가 바로
예술작품이 될 수는 없으며, 사람들이 흥미나 감흥을 느낄 때에만 비로소 ‘미적’이라고 할 수 있다. 예술의
관심이 사회의 규칙과 원리에 포함된다는 시각이나 이러한 원리를 예술에 적용하려는 시도는 미적 창조와
감상 모두에 유해한 것이다. 미적 대상은 인식적인 가치로부터 분리되어 있으며, 감각적이거나 형식적인 성
질 자체를 깨달았을 때의 순수한 기쁨에서 정당성을 얻는다. 아름다운 사물에서 ‘아름다움’의 의미를 찾아내
고 받아들이는 것은 교양 있고 선택된 사람의 특권이다. 셰익스피어의 작품은 운문(韻文)의 수려함, 비유적
표현의 다양함과 풍부함, 그리고 구성의 탁월함 때문에 상찬을 받는다. 예술가의 창조성은 그 자체로 목적이 된다.
그렇다면 예술은 무엇에 도움을 줄 수 있는가? 상업주의는 도구적 문화를 요구하며 예술의 유용성만을 강
조하기 때문에 비판에 직면하고 있다. 음미되는 가치 자체만으로 예술은 존재의 이유가 있다. 예술은 절대로
어떤 것에도 도움을 주지 않으며, 역사, 사회, 그리고 개인의 맥락과는 단절된 자율적 유기체이다. 계산될 수
있는 한에서 세계는 과학의 지배를 받기 마련이지만, 예술의 왕국은 ‘계산될 수 없는 곳’에서 시작된다. 예술
은 매 순간 자기 충만성을 향유하게 하며, 경험을 생기 있게 하고, 경험이 지닌 상상적이고 정서적인 기쁨을
즐기게 한다. 좋은 삶이란 경험의 매 순간을 풍부하고 정열적으로 사는 것이다. 그러므로 예술을 사랑하는
것이야 말로 좋은 삶의 원천이 된다.


Next, I will show you the example question. This question was generated from the sample set of prompts above.


[예시 질문]
제시문(가) ∼ 제시문(라)는 예술에 관한 견해를 담고 있다. 제시문들을 상반된 두 입장으로 분류하고 각 입장을 요약하시오.

In this way, the example question references the content of the sample prompts, summarizing them or asking about topics that can be considered from them. When creating question, it's important to consider the relationships between the contents of each prompt.
'''

instruction = '''
Please create one question and one answer based on the following prompts. You need to generate questions from the [new prompts]. When generating question, the content should not imply or suggest the answers. Please delete any unnecessary sentences. The format should be as follows:

["Question"
"Answer"]

Please create one question and one answer. As I said, when generating question, the content should not imply or suggest the answer. When creating question, it's important to consider the relationships between the contents of each prompt. The question should be in Korean, and concise, consisting of 1-2 sentences.


[new prompts (가)]
협력이 가능한 일반적인 요건은 두 가지이다. 첫째, 협력은 호혜주의를 바탕으로 한다. 둘째, 호혜주의가 오
랜 기간 동안 안정적으로 유지될 수 있다는 믿음이 있어야 한다. 결국 서로가 상대방을 도울 수 있을 때 협
력이 가능하다. 그런데 도울 때 대가를 치러야 한다면 딜레마가 생길 수 있다. 협력 당사자의 입장에서 보면
상대방의 협력 덕택에 얻는 이득이 내가 협력할 때 치러야 할 비용보다 더 커야만 협력할 이유가 생겨서 상
호 이득의 기회를 현실화할 수 있다. 이 경우 협력 당사자 모두가 상호 협력을 상호 배반보다 더 선호하게
되는 것은 당연하다. 그러나 내가 원하는 것을 얻기가 쉽지는 않다. 다음 두 이유 때문이다. 첫째, 나는 도움
을 주지 않는 것이 단기적으로 더 이득이지만 그럼에도 불구하고 상대방은 나를 돕도록 유도하고 싶어 한
다. 둘째, 나는 남에게 큰 비용이 드는 도움을 제공하지 않으면서도 받을 수 있는 도움은 모두 받고 싶은 유
혹을 느낀다. 하지만 일단 한 집단 안에서 호혜주의를 바탕으로 협력이 자리를 잡으면 어떤 비협력적인 전
략도 침범하기 어려워서 스스로를 지켜낼 수 있다.
협력이 배태될 수 있고, 온갖 다양한 전략이 뒤섞여 있는 환경에서도 번성할 수 있으며, 또 일단 자리를 잡
은 뒤에는 스스로를 보호할 수 있다는 사실은 매우 고무적이다. 하지만 무엇보다 흥미로운 사실은 이러한
결과를 얻는데 개인이나 사회적인 환경의 특성은 거의 영향을 미치지 못한다는 점이다. 개인은 논리적일 필
요가 없고 어떻게 하고 왜 그렇게 해야 하는지 알지 못해도 된다. 진화 과정은 성공적인 전략들이 자연적으
로 번성하게 해 준다. 또한 개인은 서로 메시지나 약속을 주고받을 필요도 없다. 말도 필요하지 않다. 행동
이 말을 대신하기 때문이다. 마찬가지로 개인 사이에 어떤 신뢰도 필요하지 않다. 호혜주의만으로 충분히 배
반을 비생산적이 되게 할 수 있기 때문이다. 이타주의도 필요하지 않다. 성공적인 전략은 이기주의자한테서
도 협력을 이끌어내기 때문이다.
협력의 창발과 성장, 유지에 꼭 필요한 개인과 사회 환경에 대한 가정이 몇 가지 있기는 하다. 우선 예전에
상호 작용했던 협력자를 알아볼 수 있어야 하며 상대와의 과거 상호 작용 내력을 기억할 수 있어야 한다. 그
래야 그에 따라 반응할 수 있다. 하지만 실제로는 인식과 기억의 요건을 맞추는 것이 그렇게 어려운 것은 아
니다. 박테리아조차도 단 하나의 상대하고만 상호 작용한다거나, 단 하나의 전략을 가지고 상대방의 가장 최
근 행위에 대해서만 반응하는 식으로 이런 요건을 충족시킨다. 박테리아가 할 수 있으면 사람도 할 수 있다.

[new prompts (나)]
인간의 노력을 조정하는 수단으로 경쟁의 힘을 최대한 잘 활용해야 한다. 이는 유효한 경쟁이 창출될 수
있는 곳에서는 그 어떤 방법보다도 이것이 개별적 노력의 좋은 길잡이가 된다는 확신에 기초한 것이다. 나
는 경쟁이 유익하게 작동하려면 세심하게 배려된 법적 틀이 필요하다는 사실을 부인하지 않는다. 또한 만약
경쟁이 유효하게 이루어지도록 하는 조건을 충족시키기 어려운 경우에는 다른 방법을 경제활동의 길잡이로
삼아야 한다는 것도 부정하지 않는다. 그러나 개인의 개별적 노력을 조정하는 방법으로 더욱 열등한 방법이
경쟁을 대체하는 것에는 반대한다.
그리고 알려진 방법 중 가장 효율적이라는 이유뿐만 아니라 더 크게는 권력의 강제적이고도 자의적인 간섭
없이도 우리의 행위를 조정할 수 있는 유일한 방법이기 때문에 나는 경쟁을 우월한 방법으로 간주한다. 사
실, 경쟁을 선호하는 핵심 이유 중 하나는 의식적인 사회적 통제가 필요하지 않다는 점이며, 특정한 직업이
그 직업과 연관된 불리한 점과 위험 요소를 상쇄하고도 남을 만큼 전망이 있는지 스스로 판가름할 기회를
각자에게 부여한다는 점이다. 경쟁을 사회 조직의 원칙으로 잘 활용하기 위해서는 강제적 간섭을 배제해야
하지만 그 작동을 도와줄 수 있는 특정한 방식의 간섭은 인정할 수 있으며 정부의 적절한 조치가 필요한 경
우도 있다. 그러나 강제력을 사용해서는 안 된다는 점은 특별히 강조해야 한다. 시장 참여자는 거래 상대방
을 찾을 수 있는 한 어떤 가격에서건 자유롭게 팔고 살 수 있어야 하고, 누구든 자유롭게 생산할 수 있고,
팔릴 수 있는 어떤 것도 생산하여 팔고 살 수 있어야 한다.
업종에 상관없이 모든 사람에게 같은 조건으로 진입을 자유롭게 허용해야 한다는 것은 본질적으로 중요하
다. 아울러 개인이나 단체가 공개적인 혹은 드러나지 않은 힘을 이용하여 이러한 진입을 제한하려는 것을
법이 용인하지 않아야 한다는 것 또한 본질적으로 중요하다. 특정 상품에 대해 가격이나 물량을 통제하게
되면 각자의 노력을 유효하게 조정하는 경쟁의 능력은 박탈된다. 왜냐하면 이 경우 가격의 변화가 변화된
모든 상황을 더 이상 반영하지 못하게 되며, 그 결과 가격의 변화가 더 이상 개인의 행위를 나타내는 믿을
만한 길잡이가 될 수 없기 때문이다.

[new prompts (다)]
전체 사회 구성원이 공동으로 소유한 산림과 바다 같은 공유 자원은 배제성은 없지만 경합성이 있다. 배제
성이 없으므로 대가를 지불하지 않고도 누구나 공유 자원을 소비할 수 있는 반면, 경합성이 있으므로 누군
가가 공유 자원을 소비하면 다른 사람의 소비 기회는 감소한다. 따라서 우리는 공유 자원을 다른 사람보다
먼저 더 많이 소비하려고 경쟁한다. 이와 같은 특성 때문에 공유 자원은 쉽게 남용되어 필요한 양보다 과다
하게 소비하게 되고 고갈될 가능성이 커진다. 이에 대해 일부 공동체주의자는 공동체적 해법이야말로 인간
의 본성에 가장 걸맞은 자연스러운 해법이라고 주장한다. 이들에게 자연은 “적자생존” 혹은 “피로 물든 이빨
과 발톱” 등으로 묘사되어온 생존 경쟁의 장이 아니다. 오히려 이들은 상호 부조의 감정이야말로 수십만 년
에 걸친 집단생활을 통해 그리고 지난 수천 년의 사회생활을 통해 배양된 것이며, “전쟁터에서처럼 사람들이
미쳐 돌아가는 상태가 아니라면” 상호 부조의 감정은 거스를 수 없는 법이라고 주장한다.
그러나 인간은 언제나 공공의 이득을 위해서 행동할 준비가 되어 있는 존재는 아니다. 그렇기 때문에 인간
본성에 기댈 수만은 없으며 적절한 유인이 필요하고 적절한 제도가 필요하다. 적절한 제도 하에서는 자신의
이익과 공동의 이익을 조화시킬 수 있을 것이기 때문이다. 이때 적절한 제도란 물질적 이득을 추구하는 개
인을 외적으로 그리고 위에서 아래로 강제하는 장치가 아니라 상호 신뢰를 통해 서로를 규제할 수 있는 견
제 장치이다. 이것이 가능한 것은 인간은 협조하려는 상대방의 의지가 확인되면 언제든지 이에 협조로 응답
하고, 자신의 행동이 타인에게 미치는 영향을 고려하며, 다른 누군가의 행동이 타인에게 해를 입히는 경우
자기 일처럼 나서서 이를 제어하려는 의향을 가진 존재이기 때문이다.
1960년대 오스트레일리아에서 바다가재를 잡는 지역의 어부들은 바다가재의 숫자가 계속 줄어드는 문제에
직면해 있었다. 문제를 해결하기 위해 어부들은 좋은 생각을 해냈다. 공동체를 결성해서 설치할 수 있는 어
망의 숫자를 제한하기로 한 것이다. 이에 다른 사람이 어망을 더 설치하는 것을 서로 감시하게 되면서 불법
적으로 어망을 설치하는 어민이 사라졌고 결국 바다가재의 숫자를 늘리는 데 성공했다. 만약 정부에서 일방
적으로 어망을 제한했다면 여러 나라에서 그랬던 것처럼 일부 어부가 경비정 몰래 바다가재를 제한 없이 잡
다가 경비정이 쫓아오면 잡히지 않기 위해 그물을 끊고 도망가는 불법 조업이 기승을 부렸을 것이다. 그런
데 어부들이 자율적으로 제한을 두면서 자연스럽게 불법 조업이 자취를 감추었다.
공동체를 토대로 사회질서를 유지할 수 있다는 생각을 누군가는 낡은 이념으로, 다른 누군가는 유토피아적
이념으로 여긴다. 그러나 많은 경우 공동체 구성원이 상호 감시하고 상호 제재하는 것이 공유 자원을 관리
하는 효과적인 방법이 된다. 공동체 구성원은 외적 권위체가 갖지 못하는 정보를 갖고 있으며, 공동체 내에
서 상호 신뢰를 기초로 서로를 규제해가면서 비극을 피할 수 있기 때문이다.

[new prompts (라)]
경쟁을 통해 각자의 능력이 최대한 발휘되기도 하지만 경쟁에서 진 쪽은 여러 가지 부작용을 겪기도 하는
것이 사실이다. 경쟁이란 본질적으로 ‘차가운 것’이기 때문이다. 그래서 시장에서는 경쟁에서 지지 않기 위해
경쟁관계의 두 기업이 일시적으로 협력하거나 다양한 방식의 협력관계를 경쟁 속에 부분적으로 도입하는 역
설적인 경우를 볼 수 있다. 예를 들어 오랫동안 경쟁하다가도 경쟁자에게 부품을 공급하는 협력적 공급자로
위치가 일시적으로 변화할 수 있다. 협력을 도입하여 경쟁의 형태가 간접적인 것으로 변화할 수도 있다. 예
들 들어 한 기업이 다른 두 기업과 각각 독립적으로 파트너십을 맺고 협력하여 양쪽의 파트너 기업들이 서
로 경쟁하는 구도가 만들어 질 수도 있다. 판매에서는 경쟁하지만 연구 및 개발에서는 잠정적으로 협력한다거
나 아예 부서를 나누어서 어떤 부서에서는 협력을, 다른 부서에서는 경쟁을 진행하는 방식으로 부분적 협력이
진행될 수도 있다. 특히 새로운 기술이 등장해서 기술 혹은 산업간 융합이 일어날 때, 그리고 기업 규모가
작아서 경쟁력을 높여야 할 때 경쟁자와 협력하는 현상이 일어나기도 한다. 그럼에도 경쟁 메커니즘은 근본
적으로 기업들이 경쟁에서 우위를 확보하기 위해 혁신에 더 집중하도록 유도한다. 일반적으로 기업들은 독
점보다는 경쟁 상황일 때 연구개발 등 혁신 활동에 더 많이 투자할 유인을 갖기 때문이다.
사회 전체 차원에서 보면 경쟁이 심해질수록 사회생활의 접촉대상이 되는 타인은 실재적 혹은 잠재적 경쟁
대상으로만 간주될 것이며 이러한 사고가 팽배할 때 휴머니즘을 토대로 한 사회적 친화력을 견지하기는 어
렵다. 그래서 경제사상가들은 경쟁이 국부의 증대에 기여하지만 사회적 동질성과 도덕을 잠식한다는 지적을
자주 해 왔다. 이런 문제를 고려할 때 제기되는 과제는 한편으로는 경쟁의 기능을 계속 살리면서 다른 한편
으로는 경쟁이 유발하는 비인도적인 측면을 약화시킬 수 있는 ‘황금의 중도’를 설정하는 일이다. 2차 세계대
전 이후 독일을 위시한 몇몇 서구국가들이 도입한 사회적 시장경제 제도는 바로 그와 같은 사실을 배경으로
형성된 것이었다. 효율성을 증대시키는 시장경제의 기능을 토대로 하면서 동시에 자본주의 사회에 내재하는
차가움이나 사회·경제적 역기능을 극소화하자는 것이다.
최근 들어 과거의 성장제일주의 정책을 수정하여 안정과 형평 내지는 삶의 질을 조화시키겠다고 선언하는
국가를 쉽게 찾을 수 있다. 이를 위해서는 무엇보다도 사회의 기본적 가치에 대한 공감이 형성되지 않으면
안 된다. 그렇지 않을 경우 표출되는 다양한 갈등은 사회적 제어능력을 벗어날 위험이 있기 때문이다. 공존
원리로서의 공정한 경쟁을 통한 진보를 훼손하지 않으면서도 휴머니즘을 기저로 하는 사회적 삶의 실존과
조화되는 규범을 마련하는 토론의 장에 모든 사회 구성원 혹은 이익집단의 참여가 필요한 이유도 여기에 있
다. 최근 폭발적으로 분출되는 다양한 욕구·견해·주장 등은 아마도 새로운 질서창출을 위한 사회적 공감대
형성의 불가피한 진통일 것이다.
'''

messages = [
    {"role": "system", "content": f"{PROMPT}"},
    {"role": "user", "content": f"{instruction}"}
    ]

prompt = tokenizer.apply_chat_template(
    messages,
    tokenize = False,
    add_generation_prompt=True
)

generation_kwargs = {
    "max_tokens":100000,
    "stop":["<|eot_id|>"],
    "top_p":0.9,
    "temperature":0.2,
    "echo":True, # Echo the prompt in the output
}

resonse_msg = model(prompt, **generation_kwargs)
print(resonse_msg['choices'][0]['text'][len(prompt):])

Llama.generate: prefix-match hit

llama_print_timings:        load time =     141.95 ms
llama_print_timings:      sample time =     341.74 ms /   148 runs   (    2.31 ms per token,   433.08 tokens per second)
llama_print_timings: prompt eval time =    3764.92 ms /  4833 tokens (    0.78 ms per token,  1283.69 tokens per second)
llama_print_timings:        eval time =    3851.82 ms /   147 runs   (   26.20 ms per token,    38.16 tokens per second)
llama_print_timings:       total time =    8184.22 ms /  4980 tokens


Here is the question and answer in the format that I earlier sent to you.

[질문]
예시문(가)와 제시문(라)를 바탕으로, 두 글자를 분류하고 요약하시오.

[답변]
제시문(가)와 제시문(라)를 분류하면, 제시문(라)는 '경쟁'을 강조하는 것에 중점을 두고 있고, 예시문(가)는 '공중'에서 서로 협력하는 것을 강조하고 있다. 요약하자면, 제시문(라)는 경쟁으로 인한 부작용이 존재하지만 경쟁 관계 내에서 협력과 함께 진행되어야 한다는 것에 초점을 맞추고 있으며, 예시문(가)는 상호 협력이 필요하고 이를 통해 사회적인 문제를 해결할 수 있다는 것에 강조하는 것이다.
